## Первая версия

In [ ]:
# Импорт только необходимых библиотек
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from IPython.display import display
from PIL import Image
import io

# Основной класс для нормализации изображений
class CharacterNormalizer:
    def __init__(self):
        # Загрузка каскадов Хаара для обнаружения лиц (включены в OpenCV)
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        
        # Константы для стандартизации
        self.portrait_mode = True  # Если True - обрезаем до портрета
        self.target_width = 800    # Целевая ширина выходного изображения
        self.target_height = 1200  # Целевая высота выходного изображения
        self.face_to_height_ratio = 0.2  # Лицо должно занимать примерно 20% высоты кадра
        
        # НОВЫЕ ПАРАМЕТРЫ для предотвращения обрезки головы
        self.head_padding_ratio = 0.5  # Дополнительное пространство над головой (относительно высоты лица)
        self.debug_mode = False  # Режим отладки для визуализации рамок
        
    def detect_face(self, image):
        """Определяет положение и размер лица на изображении используя OpenCV"""
        # Преобразуем в оттенки серого для обнаружения лиц
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Обнаружение лиц с разными параметрами для повышения точности
        faces = self.face_cascade.detectMultiScale(
            gray, 
            scaleFactor=1.1, 
            minNeighbors=5, 
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )
        
        if len(faces) == 0:
            # Пробуем с менее строгими параметрами
            faces = self.face_cascade.detectMultiScale(
                gray, 
                scaleFactor=1.2, 
                minNeighbors=3, 
                minSize=(20, 20),
                flags=cv2.CASCADE_SCALE_IMAGE
            )
            
        if len(faces) == 0:
            return None
        
        # Берем наибольшее обнаруженное лицо (если их несколько)
        if len(faces) > 1:
            # Выбираем лицо с наибольшей площадью
            areas = [w*h for (x, y, w, h) in faces]
            max_idx = areas.index(max(areas))
            x, y, w, h = faces[max_idx]
        else:
            # Если только одно лицо, берем его
            x, y, w, h = faces[0]
        
        return {
            'x': x,
            'y': y,
            'width': w,
            'height': h,
            'center_x': x + w // 2,
            'center_y': y + h // 2
        }
    
    def detect_object(self, image):
        """Определяет объекты на изображении, в первую очередь лица"""
        # Пытаемся обнаружить лицо
        face_data = self.detect_face(image)
        
        h, w, _ = image.shape
        
        if face_data:
            # Если нашли лицо, расширяем область, чтобы включить всю голову и тело
            x, y, width, height = face_data['x'], face_data['y'], face_data['width'], face_data['height']
            
            # ИСПРАВЛЕНИЕ: Добавляем дополнительное пространство над головой
            extra_head_space = int(height * self.head_padding_ratio)
            y = max(0, y - extra_head_space)
            height += extra_head_space
            
            # Предполагаем, что тело находится под лицом и шире его
            body_width = int(width * 3)
            body_height = int(height * 5)
            
            # Центрируем тело по лицу
            body_x = max(0, x + width//2 - body_width//2)
            body_y = min(h - 1, y + height)  # Начинаем от нижней границы лица
            
            # Убеждаемся, что не выходим за границы изображения
            body_width = min(body_width, w - body_x)
            body_height = min(body_height, h - body_y)
            
            result = {
                'x': min(x, body_x),
                'y': y,  # Верхняя граница - верх лица с учетом отступа
                'width': max(width, body_width),
                'height': max(height, body_height + (y - body_y)),
                'center_x': x + width // 2,
                'center_y': y + height // 2,
                'face_data': face_data
            }
            
            return result
        
        # Если лицо не найдено, используем более продвинутые методы обнаружения объектов
        try:
            # Преобразуем изображение для выделения объекта от фона
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            blurred = cv2.GaussianBlur(gray, (5, 5), 0)
            
            # Пробуем использовать адаптивный порог для лучшего выделения объектов
            thresh = cv2.adaptiveThreshold(
                blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                cv2.THRESH_BINARY_INV, 11, 2
            )
            
            # Находим контуры
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            if contours:
                # Берем самый большой контур
                largest_contour = max(contours, key=cv2.contourArea)
                x, y, w, h = cv2.boundingRect(largest_contour)
                
                return {
                    'x': x,
                    'y': y,
                    'width': w,
                    'height': h,
                    'center_x': x + w // 2,
                    'center_y': y + h // 2
                }
        except Exception as e:
            if self.debug_mode:
                print(f"Ошибка при поиске контуров: {e}")
        
        # Если ничего не нашли, берем всё изображение
        return {
            'x': 0,
            'y': 0,
            'width': w,
            'height': h,
            'center_x': w // 2,
            'center_y': h // 2
        }
        
    def normalize_image(self, image):
        """Нормализует изображение к стандартному формату"""
        # Определяем объект на изображении
        obj_data = self.detect_object(image)
        h, w, _ = image.shape
        
        # Если нашли лицо, используем его для масштабирования
        if 'face_data' in obj_data:
            face = obj_data['face_data']
            face_height = face['height']
            
            # Вычисляем масштаб, чтобы лицо занимало нужную долю высоты кадра
            scale = (self.target_height * self.face_to_height_ratio) / face_height
            
            # ИСПРАВЛЕНИЕ: Расчет центра смещен выше, чтобы не обрезать голову
            # Начинаем с верхней точки лица + дополнительный отступ выше
            face_top = face['y']
            
            # Верхняя граница кадра = верхняя граница лица минус отступ
            extra_head_space = int(face_height * self.head_padding_ratio)
            top_y = max(0, face_top - extra_head_space)
            
            # Лицо должно быть размещено в верхней части кадра, но не в самом верху
            # Определяем, где должен быть верх лица (примерно на уровне 0.15-0.2 от высоты кадра)
            target_face_top_position = int(self.target_height * 0.15)
            
            # Вычисляем координаты центра кадра так, чтобы лицо оказалось в верхней части
            center_x = face['center_x']
            
            # Вместо центрирования, мы позиционируем изображение относительно верха лица
            crop_height = int(self.target_height / scale)
            crop_width = int(self.target_width / scale)
            
            # Определяем координаты верхнего левого угла кадра
            x1 = max(0, center_x - crop_width // 2)
            y1 = max(0, top_y)  # Начинаем с верхней границы лица минус отступ
            
            # Корректируем x1, если кадр выходит за границу изображения справа
            if x1 + crop_width > w:
                x1 = max(0, w - crop_width)
        else:
            # Если лицо не нашли, масштабируем и центрируем по найденному объекту
            obj_height = obj_data['height']
            scale = self.target_height / (obj_height * 1.2)  # Немного меньше запаса
            
            center_x = obj_data['center_x']
            center_y = obj_data['center_y']
            
            # Вычисляем размеры вырезаемой области
            crop_width = int(self.target_width / scale)
            crop_height = int(self.target_height / scale)
            
            # Вычисляем координаты вырезаемой области
            x1 = max(0, center_x - crop_width // 2)
            y1 = max(0, center_y - crop_height // 2)
            
            # Корректируем, если выходим за границы изображения
            if x1 + crop_width > w:
                x1 = max(0, w - crop_width)
            
        # Дополнительная проверка выхода за нижнюю границу
        if y1 + crop_height > h:
            y1 = max(0, h - crop_height)
            
        # Если область выходит за границы, корректируем ее размер
        crop_width = min(crop_width, w - x1)
        crop_height = min(crop_height, h - y1)
        
        # Вырезаем область
        cropped = image[y1:y1+crop_height, x1:x1+crop_width]
        
        # Добавляем отладочную информацию, если включен режим отладки
        if self.debug_mode and 'face_data' in obj_data:
            face = obj_data['face_data']
            debug_img = cropped.copy()
            # Рисуем рамку лица относительно вырезанной области
            face_x = face['x'] - x1
            face_y = face['y'] - y1
            if face_x >= 0 and face_y >= 0:
                cv2.rectangle(debug_img, 
                             (face_x, face_y), 
                             (face_x + face['width'], face_y + face['height']), 
                             (0, 255, 0), 2)
            cropped = debug_img
        
        # Если обрезанная область не соответствует нужным пропорциям, 
        # добавляем padding нужного цвета
        if crop_width / crop_height != self.target_width / self.target_height:
            # Создаем новое изображение с нужными пропорциями
            target_ratio = self.target_width / self.target_height
            current_ratio = crop_width / crop_height
            
            if current_ratio > target_ratio:
                # Изображение слишком широкое, добавляем сверху и снизу
                new_height = int(crop_width / target_ratio)
                padding_top = (new_height - crop_height) // 2
                padding_bottom = new_height - crop_height - padding_top
                
                # Выбираем цвет фона (средний цвет по краям)
                bg_color = np.median(image[:10, :], axis=(0, 1))
                
                # Создаем новое изображение с padding
                padded = np.full((new_height, crop_width, 3), bg_color, dtype=np.uint8)
                padded[padding_top:padding_top+crop_height, :] = cropped
                cropped = padded
            else:
                # Изображение слишком высокое, добавляем по бокам
                new_width = int(crop_height * target_ratio)
                padding_left = (new_width - crop_width) // 2
                padding_right = new_width - crop_width - padding_left
                
                # Выбираем цвет фона (средний цвет по краям)
                bg_color = np.median(image[:, :10], axis=(0, 1))
                
                # Создаем новое изображение с padding
                padded = np.full((crop_height, new_width, 3), bg_color, dtype=np.uint8)
                padded[:, padding_left:padding_left+crop_width] = cropped
                cropped = padded
        
        # Изменяем размер до целевых значений
        normalized = cv2.resize(cropped, (self.target_width, self.target_height))
        
        return normalized
    
    def process_directory(self, input_dir, output_dir):
        """Обрабатывает все изображения в директории"""
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        results = []
        for filename in os.listdir(input_dir):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                try:
                    filepath = os.path.join(input_dir, filename)
                    image = cv2.imread(filepath)
                    
                    if image is None:
                        print(f"Не удалось загрузить {filename}")
                        results.append(f"Ошибка загрузки: {filename}")
                        continue
                        
                    normalized = self.normalize_image(image)
                    
                    # Сохраняем результат
                    output_path = os.path.join(output_dir, f"normalized_{filename}")
                    cv2.imwrite(output_path, normalized)
                    print(f"Обработано: {filename}")
                    results.append(f"Успешно: {filename}")
                except Exception as e:
                    print(f"Ошибка при обработке {filename}: {str(e)}")
                    results.append(f"Ошибка: {filename} - {str(e)}")
        return results

    def set_portrait_mode(self, enabled=True):
        """Включает или выключает портретный режим"""
        self.portrait_mode = enabled
        
    def set_target_dimensions(self, width, height):
        """Устанавливает целевые размеры выходного изображения"""
        self.target_width = width
        self.target_height = height
        
    def set_face_ratio(self, ratio):
        """Устанавливает, какую долю от высоты должно занимать лицо"""
        self.face_to_height_ratio = ratio
        
    def set_head_padding(self, ratio):
        """Устанавливает дополнительный отступ над головой (относительно размера лица)"""
        self.head_padding_ratio = ratio
        
    def enable_debug(self, enabled=True):
        """Включает или выключает режим отладки"""
        self.debug_mode = enabled
        
    def visualize_detection(self, image):
        """Визуализирует обнаруженные объекты на изображении"""
        img_copy = image.copy()
        
        # Обнаруживаем объекты
        face_data = self.detect_face(img_copy)
        obj_data = self.detect_object(img_copy)
        
        # Если есть лицо, рисуем его
        if face_data:
            x, y = face_data['x'], face_data['y']
            w, h = face_data['width'], face_data['height']
            cv2.rectangle(img_copy, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(img_copy, "Face", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            
            # Визуализируем область "головы" с отступом
            extra_head_space = int(h * self.head_padding_ratio)
            head_y = max(0, y - extra_head_space)
            cv2.rectangle(img_copy, (x, head_y), (x+w, y+h), (0, 255, 255), 2)
            cv2.putText(img_copy, "Head", (x, head_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)
        
        # Рисуем область объекта
        x, y = obj_data['x'], obj_data['y']
        w, h = obj_data['width'], obj_data['height']
        cv2.rectangle(img_copy, (x, y), (x+w, y+h), (0, 0, 255), 2)
        cv2.putText(img_copy, "Object", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        
        # Преобразуем из BGR в RGB для корректного отображения в Jupyter
        img_rgb = cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB)
        return img_rgb

# Функция для отображения изображений в Jupyter
def display_images(images, titles=None, figsize=(15, 10)):
    """Отображает список изображений в Jupyter Notebook"""
    n = len(images)
    if titles is None:
        titles = ['Image {}'.format(i) for i in range(1, n + 1)]
    fig, axs = plt.subplots(1, n, figsize=figsize)
    if n == 1:
        axs = [axs]
    for i in range(n):
        if len(images[i].shape) == 2:  # Если изображение в градациях серого
            axs[i].imshow(images[i], cmap='gray')
        else:  # Если изображение цветное
            if images[i].shape[2] == 3:  # Если RGB
                axs[i].imshow(images[i])
            else:  # Если BGR (OpenCV формат)
                axs[i].imshow(cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB))
        axs[i].set_title(titles[i])
        axs[i].axis('off')
    plt.tight_layout()
    plt.show()

# Пример интерактивной настройки параметров для эксперимента
def interactive_test(image_path, initial_head_padding=0.5):
    """Интерактивный тест разных параметров отступа над головой"""
    try:
        import ipywidgets as widgets
        from IPython.display import display
        
        image = cv2.imread(image_path)
        if image is None:
            print(f"Не удалось загрузить изображение: {image_path}")
            return
            
        normalizer = CharacterNormalizer()
        normalizer.set_head_padding(initial_head_padding)
        normalizer.enable_debug(True)
        
        # Создаем слайдер для настройки отступа
        head_padding_slider = widgets.FloatSlider(
            value=initial_head_padding,
            min=0.0,
            max=2.0,
            step=0.1,
            description='Отступ над головой:'
        )
        
        # Функция обновления изображения
        def update_image(head_padding):
            normalizer.set_head_padding(head_padding)
            detected = normalizer.visualize_detection(image)
            normalized = normalizer.normalize_image(image)
            normalized_rgb = cv2.cvtColor(normalized, cv2.COLOR_BGR2RGB)
            
            display_images(
                [cv2.cvtColor(image, cv2.COLOR_BGR2RGB), detected, normalized_rgb],
                ['Исходное', 'Обнаруженные объекты', f'Нормализованное (отступ={head_padding:.1f})'],
                figsize=(18, 6)
            )
        
        # Создаем интерактивный виджет
        interactive_widget = widgets.interactive(update_image, head_padding=head_padding_slider)
        display(interactive_widget)
        
    except ImportError:
        print("Для интерактивных тестов нужен пакет ipywidgets. Установите его командой: pip install ipywidgets")
        
        # Запускаем неинтерактивный тест
        normalizer = CharacterNormalizer()
        normalizer.set_head_padding(initial_head_padding)
        normalizer.enable_debug(True)
        
        image = cv2.imread(image_path)
        detected = normalizer.visualize_detection(image)
        normalized = normalizer.normalize_image(image)
        normalized_rgb = cv2.cvtColor(normalized, cv2.COLOR_BGR2RGB)
        
        display_images(
            [cv2.cvtColor(image, cv2.COLOR_BGR2RGB), detected, normalized_rgb],
            ['Исходное', 'Обнаруженные объекты', f'Нормализованное (отступ={initial_head_padding:.1f})']
        )

In [ ]:
# Нормализация PNG в портретный формат с сохранением прозрачности
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Папки для обработки
SOURCE_DIR = 'source'  # Папка с исходными изображениями
TARGET_DIR = 'final'   # Папка для сохранения результатов

# Создаем целевую папку, если она не существует
if not os.path.exists(TARGET_DIR):
    os.makedirs(TARGET_DIR)
    print(f"Создана папка: {TARGET_DIR}")

# Целевые размеры и параметры
TARGET_WIDTH = 530
TARGET_HEIGHT = 1000
FACE_RATIO = 0.15  # Размер лица относительно высоты портрета

# Функция для загрузки PNG с сохранением альфа-канала
def load_png_properly(image_path):
    # Проверяем, является ли файл PNG
    if not image_path.lower().endswith('.png'):
        return cv2.imread(image_path)
    
    # Загружаем PNG с полной информацией об альфа-канале
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    
    if image is None:
        return None
    
    # Проверяем наличие альфа-канала
    if len(image.shape) == 3 and image.shape[2] == 4:
        return image
    else:
        # Если нет альфа-канала, возвращаем обычное изображение
        return cv2.imread(image_path)

# Улучшенная функция обнаружения лица на изображениях с прозрачностью
def detect_face_in_png(image):
    """Обнаруживает лицо на изображении PNG с прозрачностью"""
    # Проверяем наличие альфа-канала
    if len(image.shape) == 3 and image.shape[2] == 4:
        # Создаем копию только RGB каналов
        rgb_image = image[:, :, :3].copy()
        # Создаем маску прозрачности (используем только непрозрачные области)
        alpha = image[:, :, 3]
        mask = alpha > 128  # Берем только пиксели с альфа > 128
        
        # Создаем временное изображение для обнаружения лица
        temp_image = rgb_image.copy()
        # Для прозрачных областей устанавливаем нейтральный цвет
        temp_image[~mask] = [200, 200, 200]  # Светло-серый фон для прозрачных областей
    else:
        # Обычное изображение без прозрачности
        temp_image = image
    
    # Загружаем каскад Хаара для обнаружения лиц
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Преобразуем в оттенки серого для обнаружения лиц
    gray = cv2.cvtColor(temp_image, cv2.COLOR_BGR2GRAY)
    
    # Обнаружение лиц с разными параметрами для повышения точности
    faces = face_cascade.detectMultiScale(
        gray, 
        scaleFactor=1.1, 
        minNeighbors=5, 
        minSize=(30, 30)
    )
    
    if len(faces) == 0:
        # Пробуем с менее строгими параметрами
        faces = face_cascade.detectMultiScale(
            gray, 
            scaleFactor=1.2, 
            minNeighbors=3, 
            minSize=(20, 20)
        )
    
    if len(faces) == 0:
        return None
    
    # Берем наибольшее обнаруженное лицо
    if len(faces) > 1:
        areas = [w*h for (x, y, w, h) in faces]
        max_idx = areas.index(max(areas))
        x, y, w, h = faces[max_idx]
    else:
        x, y, w, h = faces[0]
    
    return {
        'x': x,
        'y': y,
        'width': w,
        'height': h,
        'center_x': x + w // 2,
        'center_y': y + h // 2
    }

# Функция для определения границ непрозрачного контента
def get_content_bounds(image):
    """Находит границы непрозрачного контента в PNG"""
    # Получаем размеры изображения
    h, w = image.shape[:2]
    
    # Проверяем наличие альфа-канала
    if len(image.shape) == 3 and image.shape[2] == 4:
        alpha = image[:, :, 3]
        # Находим все непрозрачные пиксели (с альфа > 10)
        non_transparent = alpha > 10
        
        # Если нет непрозрачных пикселей, возвращаем всё изображение
        if not np.any(non_transparent):
            return (0, 0, w, h)
        
        # Находим координаты непрозрачных пикселей
        rows = np.any(non_transparent, axis=1)
        cols = np.any(non_transparent, axis=0)
        
        # Находим границы непрозрачного контента
        y_min, y_max = np.where(rows)[0][[0, -1]]
        x_min, x_max = np.where(cols)[0][[0, -1]]
        
        # Добавляем отступ для безопасности
        padding = 10
        y_min = max(0, y_min - padding)
        y_max = min(h - 1, y_max + padding)
        x_min = max(0, x_min - padding)
        x_max = min(w - 1, x_max + padding)
        
        return (x_min, y_min, x_max - x_min + 1, y_max - y_min + 1)
    else:
        # Для изображений без прозрачности возвращаем всё изображение
        return (0, 0, w, h)

# Функция для нормализации PNG в портретный формат
def normalize_png_to_portrait(image):
    """Нормализует PNG в портретный формат с сохранением прозрачности"""

    def normalize_png_to_portrait(image):
    # Добавьте эти константы в начало функции
    FACE_RATIO = 0.11      # Размер лица относительно высоты изображения
    FACE_POSITION_Y = 0.10  # Позиция лица сверху (в процентах от высоты)
    SCALE_FACTOR = 0.75     # Множитель масштаба для захвата большей части тела
    
    # Получаем размеры изображения
    h, w = image.shape[:2]
    
    # Пытаемся найти лицо
    face_data = detect_face_in_png(image)
    
    # Определяем границы контента
    x, y, content_width, content_height = get_content_bounds(image)
    
    # Создаем новое изображение с целевыми размерами и прозрачностью
    portrait = np.zeros((TARGET_HEIGHT, TARGET_WIDTH, 4), dtype=np.uint8)
    
    # Если нашли лицо, используем его для позиционирования
    if face_data:
        # Вычисляем масштаб так, чтобы лицо занимало примерно FACE_RATIO от высоты портрета
        face_height = face_data['height']
        scale = (TARGET_HEIGHT * FACE_RATIO) / face_height
        
        # Определяем позицию лица в итоговом портрете (в верхней трети)
        target_face_y = int(TARGET_HEIGHT * 0.15)  # 15% от верха
        
        # Вычисляем смещения для переноса всего изображения
        offset_x = TARGET_WIDTH // 2 - int(face_data['center_x'] * scale)
        offset_y = target_face_y - int(face_data['y'] * scale)
    else:
        # Если лицо не найдено, масштабируем по размеру контента
        # Определяем масштаб, чтобы весь контент поместился по высоте
        content_ratio = content_width / content_height
        target_ratio = TARGET_WIDTH / TARGET_HEIGHT
        
        if content_ratio > target_ratio:
            # Контент шире, масштабируем по ширине
            scale = TARGET_WIDTH / content_width
        else:
            # Контент выше, масштабируем по высоте
            scale = TARGET_HEIGHT / content_height
        
        # Центрируем контент
        offset_x = (TARGET_WIDTH - int(content_width * scale)) // 2
        offset_y = (TARGET_HEIGHT - int(content_height * scale)) // 2
    
    # Применяем масштаб к исходному изображению
    scaled_width = int(w * scale)
    scaled_height = int(h * scale)
    
    # Масштабируем только если изображение не слишком большое
    # (ограничиваем размер для экономии памяти)
    if scaled_width * scaled_height > 10000000:  # 10 мегапикселей
        # Для очень больших изображений сначала уменьшаем до разумного размера
        temp_scale = np.sqrt(10000000 / (w * h))
        temp_width = int(w * temp_scale)
        temp_height = int(h * temp_scale)
        temp_image = cv2.resize(image, (temp_width, temp_height), interpolation=cv2.INTER_AREA)
        
        # Затем масштабируем до нужного размера
        final_width = int(temp_width * (scale / temp_scale))
        final_height = int(temp_height * (scale / temp_scale))
        scaled_image = cv2.resize(temp_image, (final_width, final_height), interpolation=cv2.INTER_LANCZOS4)
    else:
        # Для изображений разумного размера масштабируем напрямую
        scaled_image = cv2.resize(image, (scaled_width, scaled_height), interpolation=cv2.INTER_LANCZOS4)
    
    # Определяем область для вставки масштабированного изображения в портрет
    # с учетом смещений
    x_start = max(0, offset_x)
    y_start = max(0, offset_y)
    x_end = min(TARGET_WIDTH, offset_x + scaled_width)
    y_end = min(TARGET_HEIGHT, offset_y + scaled_height)
    
    # Определяем область исходного изображения для копирования
    src_x_start = max(0, -offset_x)
    src_y_start = max(0, -offset_y)
    src_x_end = src_x_start + (x_end - x_start)
    src_y_end = src_y_start + (y_end - y_start)
    
    # Копируем часть масштабированного изображения в портрет с учетом прозрачности
    if len(scaled_image.shape) == 3 and scaled_image.shape[2] == 4:
        # С альфа-каналом
        scaled_section = scaled_image[src_y_start:src_y_end, src_x_start:src_x_end]
        
        # Создаем маску из альфа-канала
        alpha_mask = scaled_section[:, :, 3:4] / 255.0
        
        # Копируем RGB каналы с учетом прозрачности
        portrait[y_start:y_end, x_start:x_end, :3] = scaled_section[:, :, :3]
        portrait[y_start:y_end, x_start:x_end, 3] = scaled_section[:, :, 3]
    else:
        # Без альфа-канала (для полноты)
        portrait[y_start:y_end, x_start:x_end, :3] = scaled_image[src_y_start:src_y_end, src_x_start:src_x_end]
        portrait[y_start:y_end, x_start:x_end, 3] = 255  # Полностью непрозрачно
    
    return portrait

# Функция для сохранения изображения с прозрачностью
def save_image_with_transparency(image, output_path):
    # Для PNG сохраняем с прозрачностью
    if output_path.lower().endswith('.png'):
        if len(image.shape) == 3 and image.shape[2] == 4:
            # Если это PNG с альфа-каналом, сохраняем как есть
            cv2.imwrite(output_path, image)
        else:
            # Если нет альфа-канала, сохраняем как обычное изображение
            cv2.imwrite(output_path, image)
    else:
        # Для других форматов удаляем альфа-канал
        if len(image.shape) == 3 and image.shape[2] == 4:
            # Если есть альфа-канал, берем только RGB часть
            cv2.imwrite(output_path, image[:, :, 0:3])
        else:
            # Если нет альфа-канала, сохраняем как обычное изображение
            cv2.imwrite(output_path, image)

# Обработка всех изображений в папке
def process_all_images():
    # Получаем список всех файлов
    files = [f for f in os.listdir(SOURCE_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    if not files:
        print(f"В папке {SOURCE_DIR} не найдены изображения")
        return
    
    print(f"Найдено {len(files)} изображений для обработки")
    
    # Обрабатываем каждое изображение
    successful = 0
    failed = 0
    
    for filename in files:
        try:
            # Полный путь к исходному файлу
            input_path = os.path.join(SOURCE_DIR, filename)
            
            # Загружаем изображение с учетом прозрачности
            image = load_png_properly(input_path)
            
            if image is None:
                print(f"⚠️ Не удалось загрузить: {filename}")
                failed += 1
                continue
            
            # Для PNG с прозрачностью используем специальную обработку
            if filename.lower().endswith('.png') and len(image.shape) == 3 and image.shape[2] == 4:
                # Специальная обработка PNG в портретный формат
                processed = normalize_png_to_portrait(image)
            else:
                # Для обычных изображений используем класс CharacterNormalizer
                # (предполагается, что он определен выше)
                processed = normalizer.normalize_image(image)
            
            # Путь для сохранения результата
            output_path = os.path.join(TARGET_DIR, filename)
            
            # Сохраняем результат
            save_image_with_transparency(processed, output_path)
            
            print(f"✅ Обработано: {filename}")
            successful += 1
            
        except Exception as e:
            print(f"❌ Ошибка при обработке {filename}: {str(e)}")
            failed += 1
    
    # Итоговая статистика
    print("\n--- Результаты обработки ---")
    print(f"Всего изображений: {len(files)}")
    print(f"Успешно обработано: {successful}")
    print(f"Не удалось обработать: {failed}")
    print(f"Результаты сохранены в папку: {TARGET_DIR}")

# Запуск обработки
process_all_images()

## МЕТОД

In [2]:
# Импорт необходимых библиотек
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from IPython.display import display
from PIL import Image
import io

# Основной класс для нормализации изображений
class CharacterNormalizer:
    def __init__(self):
        # Загрузка каскадов Хаара для обнаружения лиц (включены в OpenCV)
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        
        # Константы для стандартизации
        self.portrait_mode = True  # Если True - обрезаем до портрета
        self.target_width = 800    # Целевая ширина выходного изображения
        self.target_height = 1200  # Целевая высота выходного изображения
        self.face_to_height_ratio = 0.2  # Лицо должно занимать примерно 20% высоты кадра
        
        # Новые параметры для улучшенной нормализации
        self.min_face_size = 50  # Минимальный размер лица для корректной обработки
        self.max_body_ratio = 0.8  # Максимальная доля тела от высоты изображения
        self.center_shift_threshold = 0.2  # Порог смещения центра
        
        # Параметры для предотвращения обрезки головы и боков
        self.head_padding_ratio = 0.7  # Дополнительное пространство над головой (относительно высоты лица)
        self.debug_mode = False  # Режим отладки для визуализации рамок
        
    def detect_face(self, image):
        """Определяет положение и размер лица на изображении используя OpenCV"""
        # Преобразуем в оттенки серого для обнаружения лиц
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Обнаружение лиц с разными параметрами для повышения точности
        faces = self.face_cascade.detectMultiScale(
            gray, 
            scaleFactor=1.1, 
            minNeighbors=5, 
            minSize=(self.min_face_size, self.min_face_size),
            flags=cv2.CASCADE_SCALE_IMAGE
        )
        
        if len(faces) == 0:
            # Пробуем с менее строгими параметрами
            faces = self.face_cascade.detectMultiScale(
                gray, 
                scaleFactor=1.2, 
                minNeighbors=3, 
                minSize=(20, 20),
                flags=cv2.CASCADE_SCALE_IMAGE
            )
            
        if len(faces) == 0:
            return None
        
        # Берем наибольшее обнаруженное лицо (если их несколько)
        if len(faces) > 1:
            # Выбираем лицо с наибольшей площадью
            areas = [w*h for (x, y, w, h) in faces]
            max_idx = areas.index(max(areas))
            x, y, w, h = faces[max_idx]
        else:
            # Если только одно лицо, берем его
            x, y, w, h = faces[0]
        
        return {
            'x': x,
            'y': y,
            'width': w,
            'height': h,
            'center_x': x + w // 2,
            'center_y': y + h // 2
        }
        
    def detect_object(self, image):
            """Определяет объекты на изображении, в первую очередь лица"""
            # Пытаемся обнаружить лицо
            face_data = self.detect_face(image)
            
            h, w, _ = image.shape
            
            if face_data:
                # Если нашли лицо, расширяем область, чтобы включить всю голову и тело
                x, y, width, height = face_data['x'], face_data['y'], face_data['width'], face_data['height']
                
                # Добавляем дополнительное пространство над головой
                extra_head_space = int(height * self.head_padding_ratio)
                y = max(0, y - extra_head_space)
                height += extra_head_space
                
                # Предполагаем, что тело находится под лицом и шире его
                body_width = int(width * 3)
                body_height = int(height * 5)
                
                # Центрируем тело по лицу
                body_x = max(0, x + width//2 - body_width//2)
                body_y = min(h - 1, y + height)  # Начинаем от нижней границы лица
                
                # Убеждаемся, что не выходим за границы изображения
                body_width = min(body_width, w - body_x)
                body_height = min(body_height, h - body_y)
                
                # Проверяем размер обнаруженного объекта
                result = {
                    'x': min(x, body_x),
                    'y': y,  # Верхняя граница - верх лица с учетом отступа
                    'width': max(width, body_width),
                    'height': max(height, body_height + (y - body_y)),
                    'center_x': x + width // 2,
                    'center_y': y + height // 2,
                    'face_data': face_data
                }
                
                # Проверяем адекватность обнаруженного объекта
                if (result['height'] / h > self.max_body_ratio or 
                    result['height'] / h < 0.2):
                    # Возвращаем данные по всему изображению
                    return {
                        'x': 0,
                        'y': 0,
                        'width': w,
                        'height': h,
                        'center_x': w // 2,
                        'center_y': h // 2
                    }
                
                return result
            
            # Если лицо не найдено, используем более продвинутые методы обнаружения объектов
            try:
                # Преобразуем изображение для выделения объекта от фона
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                blurred = cv2.GaussianBlur(gray, (5, 5), 0)
                
                # Пробуем использовать адаптивный порог для лучшего выделения объектов
                thresh = cv2.adaptiveThreshold(
                    blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                    cv2.THRESH_BINARY_INV, 11, 2
                )
                
                # Находим контуры
                contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                if contours:
                    # Берем самый большой контур
                    largest_contour = max(contours, key=cv2.contourArea)
                    x, y, w_cont, h_cont = cv2.boundingRect(largest_contour)
                    
                    # Проверяем размер обнаруженного объекта
                    if (h_cont / h > self.max_body_ratio or 
                        h_cont / h < 0.2):
                        # Возвращаем данные по всему изображению
                        return {
                            'x': 0,
                            'y': 0,
                            'width': w,
                            'height': h,
                            'center_x': w // 2,
                            'center_y': h // 2
                        }
                    
                    return {
                        'x': x,
                        'y': y,
                        'width': w_cont,
                        'height': h_cont,
                        'center_x': x + w_cont // 2,
                        'center_y': y + h_cont // 2
                    }
            except Exception as e:
                if self.debug_mode:
                    print(f"Ошибка при поиске контуров: {e}")
            
            # Если ничего не нашли, берем всё изображение
            return {
                'x': 0,
                'y': 0,
                'width': w,
                'height': h,
                'center_x': w // 2,
                'center_y': h // 2
            }
        
    def _is_off_center(self, obj_data, width, height):
        """Проверяет, насколько объект смещен от центра"""
        center_x = width // 2
        center_y = height // 2
        
        x_shift = abs(obj_data['center_x'] - center_x) / width
        y_shift = abs(obj_data['center_y'] - center_y) / height
        
        return x_shift > self.center_shift_threshold or y_shift > self.center_shift_threshold
    
    def normalize_image(self, image, force_center=False):
        """Нормализует изображение к стандартному формату с улучшенным сохранением краев"""
        # Определяем объект на изображении
        obj_data = self.detect_object(image)
        h, w, _ = image.shape
        
        # Принудительное центрирование, если объект сильно смещен
        if force_center or self._is_off_center(obj_data, w, h):
            obj_data['x'] = max(0, obj_data['center_x'] - obj_data['width'] // 2)
            obj_data['y'] = max(0, obj_data['center_y'] - obj_data['height'] // 2)
        
        # Сначала пытаемся выполнить сегментацию для отделения объекта от фона
        try:
            # Простая сегментация на основе порогового значения
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            blurred = cv2.GaussianBlur(gray, (7, 7), 0)
            _, mask = cv2.threshold(blurred, 20, 255, cv2.THRESH_BINARY)
            
            # Находим контуры в маске
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # Если нашли контуры, используем самый большой для определения границ
            if contours:
                largest_contour = max(contours, key=cv2.contourArea)
                x, y, w_cont, h_cont = cv2.boundingRect(largest_contour)
                
                # Если контур занимает значительную часть изображения, используем его
                if w_cont * h_cont > 0.1 * w * h:
                    # Расширяем границы для уверенности
                    silhouette_x = max(0, x - 60)
                    silhouette_width = min(w, w_cont + 60)  # Добавляем отступ с обеих сторон
                    
                    # Обновляем информацию о ширине объекта
                    obj_data['width'] = max(obj_data['width'], silhouette_width)
                    obj_data['x'] = min(obj_data['x'], silhouette_x)
        except:
            pass  # Если сегментация не удалась, продолжаем с исходными данными
        
        # Если нашли лицо, используем его для масштабирования
        if 'face_data' in obj_data:
            face = obj_data['face_data']
            face_height = face['height']
            
            # Вычисляем масштаб, чтобы лицо занимало нужную долю высоты кадра
            scale = (self.target_height * self.face_to_height_ratio) / face_height
            
            # Расчет центра смещен выше, чтобы не обрезать голову
            face_top = face['y']
            
            # Верхняя граница кадра = верхняя граница лица минус отступ
            extra_head_space = int(face_height * self.head_padding_ratio)
            top_y = max(0, face_top - extra_head_space)
            
            # Вычисляем оптимальную ширину кадра
            crop_height = int(self.target_height / scale)
            
            # Увеличиваем ширину на основе соотношения сторон
            target_ratio = self.target_width / self.target_height
            crop_width = int(crop_height * target_ratio)
            
            # Убедимся, что ширина достаточная, чтобы вместить лицо + дополнительное пространство
            min_width = int(face['width'] * 3)  # Минимальная ширина должна быть в 3 раза больше лица
            crop_width = max(crop_width, min_width)
            
            # Центрируем по центру лица, но с учетом границ объекта
            center_x = face['center_x']
            
            # Смещаем центр в сторону центра объекта
            object_center_x = obj_data['center_x'] 
            center_x = int(0.2 * face['center_x'] + 0.8 * object_center_x)
            
            # Определяем координаты верхнего левого угла кадра
            x1 = max(0, center_x - crop_width // 2)
            y1 = top_y  # Начинаем с верхней границы лица минус отступ
            
            # Дополнительная проверка, чтобы силуэт не выходил за границы кадра
            if x1 + crop_width < obj_data['x'] + obj_data['width']:
                x1 = min(w - crop_width, obj_data['x'] + obj_data['width'] - crop_width + 30)
            
            if obj_data['x'] < x1:
                x1 = max(0, obj_data['x'] - 30)
        else:
            # Если лицо не нашли, масштабируем и центрируем по найденному объекту
            obj_height = obj_data['height']
            scale = self.target_height / (obj_height * 1.2)  # Немного меньше запаса
            
            center_x = obj_data['center_x']
            center_y = obj_data['center_y']
            
            # Вычисляем размеры вырезаемой области
            crop_width = int(self.target_width / scale)
            crop_height = int(self.target_height / scale)
            
            # Вычисляем координаты вырезаемой области
            x1 = max(0, center_x - crop_width // 2)
            y1 = max(0, center_y - crop_height // 2)
        
        # Корректируем, если выходим за границы изображения
        if x1 + crop_width > w:
            x1 = max(0, w - crop_width)
            
        # Дополнительная проверка выхода за нижнюю границу
        if y1 + crop_height > h:
            y1 = max(0, h - crop_height)
            
        # Если область выходит за границы, корректируем ее размер
        crop_width = min(crop_width, w - x1)
        crop_height = min(crop_height, h - y1)
        
        # Вырезаем область
        cropped = image[y1:y1+crop_height, x1:x1+crop_width]
        
        # Добавляем отладочную информацию, если включен режим отладки
        if self.debug_mode and 'face_data' in obj_data:
            face = obj_data['face_data']
            debug_img = cropped.copy()
            # Рисуем рамку лица относительно вырезанной области
            face_x = face['x'] - x1
            face_y = face['y'] - y1
            if face_x >= 0 and face_y >= 0 and face_x < crop_width and face_y < crop_height:
                cv2.rectangle(debug_img, 
                             (face_x, face_y), 
                             (min(crop_width, face_x + face['width']), min(crop_height, face_y + face['height'])), 
                             (0, 255, 0), 2)
            cropped = debug_img
        
        # Если обрезанная область не соответствует нужным пропорциям, 
        # добавляем padding нужного цвета
        if crop_width / crop_height != self.target_width / self.target_height:
            # Создаем новое изображение с нужными пропорциями
            target_ratio = self.target_width / self.target_height
            current_ratio = crop_width / crop_height
            
            if current_ratio > target_ratio:
                # Изображение слишком широкое, добавляем сверху и снизу
                new_height = int(crop_width / target_ratio)
                padding_top = (new_height - crop_height) // 2
                padding_bottom = new_height - crop_height - padding_top
                
                # Выбираем цвет фона (средний цвет по краям)
                bg_color = np.median(image[:10, :], axis=(0, 1))
                
                # Создаем новое изображение с padding
                padded = np.full((new_height, crop_width, 3), bg_color, dtype=np.uint8)
                padded[padding_top:padding_top+crop_height, :] = cropped
                cropped = padded
            else:
                # Изображение слишком высокое, добавляем по бокам
                new_width = int(crop_height * target_ratio)
                padding_left = (new_width - crop_width) // 2
                padding_right = new_width - crop_width - padding_left
                
                # Выбираем цвет фона (средний цвет по краям)
                bg_color = np.median(image[:, :10], axis=(0, 1))
                
                # Создаем новое изображение с padding
                padded = np.full((crop_height, new_width, 3), bg_color, dtype=np.uint8)
                padded[:, padding_left:padding_left+crop_width] = cropped
                cropped = padded
        
        # Изменяем размер до целевых значений
        normalized = cv2.resize(cropped, (self.target_width, self.target_height))
        
        return normalized

    def manual_normalize(self, image, x=None, y=None, width=None, height=None):
        """
        Ручная нормализация с возможностью указать область
        """
        h, w, _ = image.shape
        
        x = x or 0
        y = y or 0
        width = width or w
        height = height or h
        
        # Обрезаем и масштабируем указанную область
        cropped = image[y:y+height, x:x+width]
        normalized = cv2.resize(cropped, (self.target_width, self.target_height))
        
        return normalized

    def process_directory(self, input_dir, output_dir):
        """Обрабатывает все изображения в директории с улучшенной обработкой"""
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        results = []
        for filename in os.listdir(input_dir):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                try:
                    filepath = os.path.join(input_dir, filename)
                    image = cv2.imread(filepath)
                    
                    if image is None:
                        print(f"Не удалось загрузить {filename}")
                        results.append(f"Ошибка загрузки: {filename}")
                        continue
                    
                    # Пытаемся нормализовать с принудительным центрированием
                    try:
                        normalized = self.normalize_image(image, force_center=True)
                    except Exception as force_center_error:
                        # Если не получилось с принудительным центрированием, 
                        # пробуем стандартную нормализацию
                        print(f"Ошибка принудительного центрирования для {filename}: {force_center_error}")
                        normalized = self.normalize_image(image)
                    
                    # Сохраняем результат
                    output_path = os.path.join(output_dir, f"normalized_{filename}")
                    cv2.imwrite(output_path, normalized)
                    print(f"Обработано: {filename}")
                    results.append(f"Успешно: {filename}")
                    
                except Exception as e:
                    print(f"Ошибка при обработке {filename}: {str(e)}")
                    results.append(f"Ошибка: {filename} - {str(e)}")
        return results

    def set_portrait_mode(self, enabled=True):
        """Включает или выключает портретный режим"""
        self.portrait_mode = enabled
        
    def set_target_dimensions(self, width, height):
        """Устанавливает целевые размеры выходного изображения"""
        self.target_width = width
        self.target_height = height
        
    def set_face_ratio(self, ratio):
        """Устанавливает, какую долю от высоты должно занимать лицо"""
        self.face_to_height_ratio = ratio
        
    def set_head_padding(self, ratio):
        """Устанавливает дополнительный отступ над головой (относительно размера лица)"""
        self.head_padding_ratio = ratio
        
    def enable_debug(self, enabled=True):
        """Включает или выключает режим отладки"""
        self.debug_mode = enabled
        
    def visualize_detection(self, image):
        """Визуализирует обнаруженные объекты на изображении"""
        img_copy = image.copy()
        
        # Обнаруживаем объекты
        face_data = self.detect_face(img_copy)
        obj_data = self.detect_object(img_copy)
        
        # Если есть лицо, рисуем его
        if face_data:
            x, y = face_data['x'], face_data['y']
            w, h = face_data['width'], face_data['height']
            cv2.rectangle(img_copy, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(img_copy, "Face", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            
            # Визуализируем область "головы" с отступом
            extra_head_space = int(h * self.head_padding_ratio)
            head_y = max(0, y - extra_head_space)
            cv2.rectangle(img_copy, (x, head_y), (x+w, y+h), (0, 255, 255), 2)
            cv2.putText(img_copy, "Head", (x, head_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)
        
        # Рисуем область объекта
        x, y = obj_data['x'], obj_data['y']
        w, h = obj_data['width'], obj_data['height']
        cv2.rectangle(img_copy, (x, y), (x+w, y+h), (0, 0, 255), 2)
        cv2.putText(img_copy, "Object", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        
        # Преобразуем из BGR в RGB для корректного отображения в Jupyter
        img_rgb = cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB)
        return img_rgb

# Утилитарные функции
def display_images(images, titles=None, figsize=(15, 10)):
    """Отображает список изображений в Jupyter Notebook"""
    n = len(images)
    if titles is None:
        titles = ['Image {}'.format(i) for i in range(1, n + 1)]
    fig, axs = plt.subplots(1, n, figsize=figsize)
    if n == 1:
        axs = [axs]
    for i in range(n):
        if len(images[i].shape) == 2:  # Если изображение в градациях серого
            axs[i].imshow(images[i], cmap='gray')
        else:  # Если изображение цветное
            if images[i].shape[2] == 3:  # Если RGB
                axs[i].imshow(images[i])
            else:  # Если BGR (OpenCV формат)
                axs[i].imshow(cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB))
        axs[i].set_title(titles[i])
        axs[i].axis('off')
    plt.tight_layout()
    plt.show()

# Пример интерактивной настройки параметров для эксперимента
def interactive_test(image_path, initial_head_padding=0.7):
    """Интерактивный тест разных параметров отступа над головой"""
    try:
        import ipywidgets as widgets
        from IPython.display import display
        
        image = cv2.imread(image_path)
        if image is None:
            print(f"Не удалось загрузить изображение: {image_path}")
            return
            
        normalizer = CharacterNormalizer()
        normalizer.set_head_padding(initial_head_padding)
        normalizer.enable_debug(True)
        
        # Создаем слайдер для настройки отступа
        head_padding_slider = widgets.FloatSlider(
            value=initial_head_padding,
            min=0.0,
            max=2.0,
            step=0.1,
            description='Отступ над головой:'
        )
        
        # Функция обновления изображения
        def update_image(head_padding):
            normalizer.set_head_padding(head_padding)
            detected = normalizer.visualize_detection(image)
            normalized = normalizer.normalize_image(image)
            normalized_rgb = cv2.cvtColor(normalized, cv2.COLOR_BGR2RGB)
            
            display_images(
                [cv2.cvtColor(image, cv2.COLOR_BGR2RGB), detected, normalized_rgb],
                ['Исходное', 'Обнаруженные объекты', f'Нормализованное (отступ={head_padding:.1f})'],
                figsize=(18, 6)
            )
        
        # Создаем интерактивный виджет
        interactive_widget = widgets.interactive(update_image, head_padding=head_padding_slider)
        display(interactive_widget)
        
    except ImportError:
        print("Для интерактивных тестов нужен пакет ipywidgets. Установите его командой: pip install ipywidgets")
        
        # Запускаем неинтерактивный тест
        normalizer = CharacterNormalizer()
        normalizer.set_head_padding(initial_head_padding)
        normalizer.enable_debug(True)
        
        image = cv2.imread(image_path)
        detected = normalizer.visualize_detection(image)
        normalized = normalizer.normalize_image(image)
        normalized_rgb = cv2.cvtColor(normalized, cv2.COLOR_BGR2RGB)
        
        display_images(
            [cv2.cvtColor(image, cv2.COLOR_BGR2RGB), detected, normalized_rgb],
            ['Исходное', 'Обнаруженные объекты', f'Нормализованное (отступ={initial_head_padding:.1f})']
        )

## Пакетная обработка

In [27]:
# Нормализация PNG в портретный формат с сохранением прозрачности
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Папки для обработки
SOURCE_DIR = 'source'  # Папка с исходными изображениями
TARGET_DIR = 'final'   # Папка для сохранения результатов

# Создаем целевую папку, если она не существует
if not os.path.exists(TARGET_DIR):
    os.makedirs(TARGET_DIR)
    print(f"Создана папка: {TARGET_DIR}")

# Целевые размеры и параметры
TARGET_WIDTH = 1000
TARGET_HEIGHT = 1500
FACE_RATIO = 0.15  # Размер лица относительно высоты портрета

# Функция для загрузки PNG с сохранением альфа-канала
def load_png_properly(image_path):
    # Проверяем, является ли файл PNG
    if not image_path.lower().endswith('.png'):
        return cv2.imread(image_path)
    
    # Загружаем PNG с полной информацией об альфа-канале
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    
    if image is None:
        return None
    
    # Проверяем наличие альфа-канала
    if len(image.shape) == 3 and image.shape[2] == 4:
        return image
    else:
        # Если нет альфа-канала, возвращаем обычное изображение
        return cv2.imread(image_path)

# Улучшенная функция обнаружения лица на изображениях с прозрачностью
def detect_face_in_png(image, min_face_size=45):
    """Обнаруживает лицо на изображении PNG с прозрачностью"""
    # Загружаем каскад Хаара для обнаружения лиц
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Проверяем наличие альфа-канала
    if len(image.shape) == 3 and image.shape[2] == 4:
        # Создаем копию только RGB каналов
        rgb_image = image[:, :, :3].copy()
        # Создаем маску прозрачности (используем только непрозрачные области)
        alpha = image[:, :, 3]
        mask = alpha > 128  # Берем только пиксели с альфа > 128
        
        # Создаем временное изображение для обнаружения лица
        temp_image = rgb_image.copy()
        # Для прозрачных областей устанавливаем нейтральный цвет
        temp_image[~mask] = [200, 200, 200]  # Светло-серый фон для прозрачных областей
    else:
        # Обычное изображение без прозрачности
        temp_image = image
    
    # Преобразуем в оттенки серого для обнаружения лиц
    gray = cv2.cvtColor(temp_image, cv2.COLOR_BGR2GRAY)
    
    # Обнаружение лиц с разными параметрами для повышения точности
    faces = face_cascade.detectMultiScale(
        gray, 
        scaleFactor=1.1, 
        minNeighbors=5, 
        minSize=(min_face_size, min_face_size)
    )
    
    if len(faces) == 0:
        # Пробуем с менее строгими параметрами
        faces = face_cascade.detectMultiScale(
            gray, 
            scaleFactor=1.2, 
            minNeighbors=3, 
            minSize=(20, 20)
        )
    
    if len(faces) == 0:
        return None
    
    # Берем наибольшее обнаруженное лицо
    if len(faces) > 1:
        areas = [w*h for (x, y, w, h) in faces]
        max_idx = areas.index(max(areas))
        x, y, w, h = faces[max_idx]
    else:
        x, y, w, h = faces[0]
    
    return {
        'x': x,
        'y': y,
        'width': w,
        'height': h,
        'center_x': x + w // 2,
        'center_y': y + h // 2
    }

# Функция для определения границ непрозрачного контента
def get_content_bounds(image):
    """Находит границы непрозрачного контента в PNG"""
    # Получаем размеры изображения
    h, w = image.shape[:2]
    
    # Проверяем наличие альфа-канала
    if len(image.shape) == 3 and image.shape[2] == 4:
        alpha = image[:, :, 3]
        # Находим все непрозрачные пиксели (с альфа > 10)
        non_transparent = alpha > 10
        
        # Если нет непрозрачных пикселей, возвращаем всё изображение
        if not np.any(non_transparent):
            return (0, 0, w, h)
        
        # Находим координаты непрозрачных пикселей
        rows = np.any(non_transparent, axis=1)
        cols = np.any(non_transparent, axis=0)
        
        # Находим границы непрозрачного контента
        y_min, y_max = np.where(rows)[0][[0, -1]]
        x_min, x_max = np.where(cols)[0][[0, -1]]
        
        # Добавляем отступ для безопасности
        padding = 10
        y_min = max(0, y_min - padding)
        y_max = min(h - 1, y_max + padding)
        x_min = max(0, x_min - padding)
        x_max = min(w - 1, x_max + padding)
        
        return (x_min, y_min, x_max - x_min + 1, y_max - y_min + 1)
    else:
        # Для изображений без прозрачности возвращаем всё изображение
        return (0, 0, w, h)

# Улучшенная функция нормализации PNG в портретный формат
def normalize_png_to_portrait(image, 
                               target_width=1000, 
                               target_height=1500, 
                               face_ratio=0.15, 
                               face_position_y=0.15, 
                               scale_factor=0.75,
                               min_face_size=45):
    """Нормализует PNG в портретный формат с сохранением прозрачности"""
    # Получаем размеры изображения
    h, w = image.shape[:2]
    
    # Пытаемся найти лицо с заданным минимальным размером
    face_data = detect_face_in_png(image, min_face_size)
    
    # Определяем границы контента
    x, y, content_width, content_height = get_content_bounds(image)
    
    # Создаем новое изображение с целевыми размерами и прозрачностью
    portrait = np.zeros((target_height, target_width, 4), dtype=np.uint8)
    
    # Если нашли лицо, используем его для позиционирования
    if face_data:
        # Вычисляем масштаб так, чтобы лицо занимало примерно заданную долю высоты портрета
        face_height = face_data['height']
        scale = (target_height * face_ratio) / face_height
        
        # Определяем позицию лица в итоговом портрете 
        target_face_y = int(target_height * face_position_y)
        
        # Вычисляем смещения для переноса всего изображения
        offset_x = target_width // 2 - int(face_data['center_x'] * scale)
        offset_y = target_face_y - int(face_data['y'] * scale)
    else:
        # Если лицо не найдено, масштабируем по размеру контента
        content_ratio = content_width / content_height
        target_ratio = target_width / target_height
        
        if content_ratio > target_ratio:
            # Контент шире, масштабируем по ширине
            scale = target_width / content_width
        else:
            # Контент выше, масштабируем по высоте
            scale = target_height / content_height
        
        # Центрируем контент
        offset_x = (target_width - int(content_width * scale)) // 2
        offset_y = (target_height - int(content_height * scale)) // 2
    
    # Применяем масштаб к исходному изображению
    scaled_width = int(w * scale)
    scaled_height = int(h * scale)
    
    # Безопасное масштабирование больших изображений
    if scaled_width * scaled_height > 10000000:  # 10 мегапикселей
        # Для очень больших изображений сначала уменьшаем до разумного размера
        temp_scale = np.sqrt(10000000 / (w * h))
        temp_width = int(w * temp_scale)
        temp_height = int(h * temp_scale)
        temp_image = cv2.resize(image, (temp_width, temp_height), interpolation=cv2.INTER_AREA)
        
        # Затем масштабируем до нужного размера
        final_width = int(temp_width * (scale / temp_scale))
        final_height = int(temp_height * (scale / temp_scale))
        scaled_image = cv2.resize(temp_image, (final_width, final_height), interpolation=cv2.INTER_LANCZOS4)
    else:
        # Для изображений разумного размера масштабируем напрямую
        scaled_image = cv2.resize(image, (scaled_width, scaled_height), interpolation=cv2.INTER_LANCZOS4)
    
    # Определяем область для вставки масштабированного изображения в портрет
    x_start = max(0, offset_x)
    y_start = max(0, offset_y)
    x_end = min(target_width, offset_x + scaled_width)
    y_end = min(target_height, offset_y + scaled_height)
    
    # Определяем область исходного изображения для копирования
    src_x_start = max(0, -offset_x)
    src_y_start = max(0, -offset_y)
    src_x_end = src_x_start + (x_end - x_start)
    src_y_end = src_y_start + (y_end - y_start)
    
    # Копируем часть масштабированного изображения в портрет с учетом прозрачности
    if len(scaled_image.shape) == 3 and scaled_image.shape[2] == 4:
        # С альфа-каналом
        scaled_section = scaled_image[src_y_start:src_y_end, src_x_start:src_x_end]
        
        # Создаем маску из альфа-канала
        alpha_mask = scaled_section[:, :, 3:4] / 255.0
        
        # Копируем RGB каналы с учетом прозрачности
        portrait[y_start:y_end, x_start:x_end, :3] = scaled_section[:, :, :3]
        portrait[y_start:y_end, x_start:x_end, 3] = scaled_section[:, :, 3]
    else:
        # Без альфа-канала (для полноты)
        portrait[y_start:y_end, x_start:x_end, :3] = scaled_image[src_y_start:src_y_end, src_x_start:src_x_end]
        portrait[y_start:y_end, x_start:x_end, 3] = 255  # Полностью непрозрачно
    
    return portrait

# Функция для сохранения изображения с прозрачностью
def save_image_with_transparency(image, output_path):
    # Для PNG сохраняем с прозрачностью
    if output_path.lower().endswith('.png'):
        if len(image.shape) == 3 and image.shape[2] == 4:
            # Если это PNG с альфа-каналом, сохраняем как есть
            cv2.imwrite(output_path, image, [cv2.IMWRITE_PNG_COMPRESSION, 9])
        else:
            # Если нет альфа-канала, сохраняем как обычное изображение
            cv2.imwrite(output_path, image)
    else:
        # Для других форматов удаляем альфа-канал
        if len(image.shape) == 3 and image.shape[2] == 4:
            # Если есть альфа-канал, берем только RGB часть
            cv2.imwrite(output_path, image[:, :, 0:3])
        else:
            # Если нет альфа-канала, сохраняем как обычное изображение
            cv2.imwrite(output_path, image)

# Обработка всех изображений в папке с расширенными параметрами
def process_all_images(
    source_dir='source', 
    target_dir='final', 
    target_width=530, 
    target_height=1000, 
    face_ratio=0.33, 
    face_position_y=0.46, 
    scale_factor=0.7,
    min_face_size=40
):
    """
    Пакетная обработка изображений с расширенными параметрами
    
    :param source_dir: Папка с исходными изображениями
    :param target_dir: Папка для сохранения результатов
    :param target_width: Целевая ширина портрета
    :param target_height: Целевая высота портрета
    :param face_ratio: Размер лица относительно высоты изображения
    :param face_position_y: Позиция лица сверху (в процентах от высоты)
    :param scale_factor: Множитель масштаба для захвата большей части тела
    :param min_face_size: Минимальный размер лица для обнаружения

    """
    # Создаем целевую папку, если она не существует
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
        print(f"Создана папка: {target_dir}")

    # Получаем список всех файлов
    files = [f for f in os.listdir(source_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    if not files:
        print(f"В папке {source_dir} не найдены изображения")
        return
    
    print(f"Найдено {len(files)} изображений для обработки")
    
    # Обрабатываем каждое изображение
    successful = 0
    failed = 0
    
    for filename in files:
        try:
            # Полный путь к исходному файлу
            input_path = os.path.join(source_dir, filename)
            
            # Загружаем изображение с учетом прозрачности
            image = load_png_properly(input_path)
            
            if image is None:
                print(f"⚠️ Не удалось загрузить: {filename}")
                failed += 1
                continue
            
            # Для PNG с прозрачностью используем специальную обработку
            if filename.lower().endswith('.png') and len(image.shape) == 3 and image.shape[2] == 4:
                # Специальная обработка PNG в портретный формат
                processed = normalize_png_to_portrait(
                    image, 
                    target_width=target_width, 
                    target_height=target_height, 
                    face_ratio=face_ratio, 
                    face_position_y=face_position_y,
                    scale_factor=scale_factor,
                    min_face_size=min_face_size
                )
            else:
                # Для обычных изображений используем стандартную обработку
                processed = normalize_png_to_portrait(
                    image, 
                    target_width=target_width, 
                    target_height=target_height, 
                    face_ratio=face_ratio, 
                    face_position_y=face_position_y,
                    scale_factor=scale_factor,
                    min_face_size=min_face_size
                )
            
            # Путь для сохранения результата
            output_path = os.path.join(target_dir, filename)
            
            # Сохраняем результат
            save_image_with_transparency(processed, output_path)
            
            print(f"✅ Обработано: {filename}")
            successful += 1
            
        except Exception as e:
            print(f"❌ Ошибка при обработке {filename}: {str(e)}")
            failed += 1
    
    # Итоговая статистика
    print("\n--- Результаты обработки ---")
    print(f"Всего изображений: {len(files)}")
    print(f"Успешно обработано: {successful}")
    print(f"Не удалось обработать: {failed}")
    print(f"Результаты сохранены в папку: {target_dir}")

# Пример использования с настройкой параметров
def interactive_test(image_path):
    """Интерактивный тест нормализации изображения"""
    try:
        import ipywidgets as widgets
        from IPython.display import display
        
        # Загружаем изображение
        image = load_png_properly(image_path)
        if image is None:
            print(f"Не удалось загрузить изображение: {image_path}")
            return
        
        # Создаем виджеты для настройки параметров
        face_ratio_slider = widgets.FloatSlider(
            value=0.15,
            min=0.05,
            max=0.5,
            step=0.01,
            description='Размер лица:'
        )
        
        face_position_slider = widgets.FloatSlider(
            value=0.15,
            min=0.05,
            max=0.5,
            step=0.01,
            description='Позиция лица:'
        )
        
        min_face_size_slider = widgets.IntSlider(
            value=50,
            min=5,
            max=100,
            step=5,
            description='Мин. размер лица:'
        )
        
        # Функция обновления изображения
        def update_image(face_ratio, face_position, min_face_size):
            # Нормализуем изображение с новыми параметрами
            processed = normalize_png_to_portrait(
                image, 
                face_ratio=face_ratio, 
                face_position_y=face_position,
                min_face_size=min_face_size
            )
            
            # Отображаем оригинал и обработанное изображение
            plt.figure(figsize=(16, 6))
            
            plt.subplot(1, 2, 1)
            if len(image.shape) == 3 and image.shape[2] == 4:
                plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGRA2RGBA))
            else:
                plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            plt.title('Оригинал')
            plt.axis('off')
            
            plt.subplot(1, 2, 2)
            plt.imshow(cv2.cvtColor(processed, cv2.COLOR_BGRA2RGBA))
            plt.title(f'Нормализовано (лицо={face_ratio:.2f}, позиция={face_position:.2f})')
            plt.axis('off')
            
            plt.tight_layout()
            plt.show()
        
        # Создаем интерактивный виджет
        interactive_widget = widgets.interactive(
            update_image, 
            face_ratio=face_ratio_slider,
            face_position=face_position_slider,
            min_face_size=min_face_size_slider
        )
        display(interactive_widget)
        
    except ImportError:
        print("Для интерактивных тестов нужен пакет ipywidgets. Установите его командой: pip install ipywidgets")
        
        # Неинтерактивный тест
        processed = normalize_png_to_portrait(image)
        
        plt.figure(figsize=(16, 6))
        
        plt.subplot(1, 2, 1)
        if len(image.shape) == 3 and image.shape[2] == 4:
            plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGRA2RGBA))
        else:
            plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title('Оригинал')
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(processed, cv2.COLOR_BGRA2RGBA))
        plt.title('Нормализовано')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()

# Запуск обработки всех изображений
if __name__ == "__main__":
    process_all_images()
    #interactive_test("source/whiskers.png")

Найдено 1 изображений для обработки
✅ Обработано: whiskers.png

--- Результаты обработки ---
Всего изображений: 1
Успешно обработано: 1
Не удалось обработать: 0
Результаты сохранены в папку: final


interactive(children=(FloatSlider(value=0.15, description='Размер лица:', max=0.5, min=0.05, step=0.01), Float…

## ИНТЕРФЕЙС

In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox
from ipywidgets import FloatSlider, IntSlider, Dropdown, Output, Text, Checkbox
from IPython.display import display, clear_output
import glob

# Папки для обработки
SOURCE_DIR = 'source'  # Папка с исходными изображениями
TARGET_DIR = 'final'   # Папка для сохранения результатов

# Создаем целевую папку, если она не существует
if not os.path.exists(TARGET_DIR):
    os.makedirs(TARGET_DIR)
    print(f"Создана папка: {TARGET_DIR}")

# Получаем список изображений в исходной папке
def get_image_list():
    images = []
    for ext in ['png', 'jpg', 'jpeg']:
        images.extend(glob.glob(os.path.join(SOURCE_DIR, f'*.{ext}')))
    return sorted([os.path.basename(img) for img in images])

# Функция для загрузки PNG с сохранением альфа-канала
def load_image_with_alpha(image_path):
    # Проверяем, является ли файл PNG
    if image_path.lower().endswith('.png'):
        # Загружаем с альфа-каналом
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        
        if image is None:
            return None
            
        # Проверяем наличие альфа-канала
        if len(image.shape) == 3 and image.shape[2] == 4:
            return image
    
    # Для других форматов или PNG без прозрачности загружаем обычным способом
    return cv2.imread(image_path)

# Функция для обнаружения лица на изображении
def detect_face(image):
    """Определяет положение и размер лица на изображении используя OpenCV"""
    # Проверяем наличие альфа-канала
    if len(image.shape) == 3 and image.shape[2] == 4:
        # Создаем копию только RGB каналов
        rgb_image = image[:, :, :3].copy()
        # Создаем маску прозрачности (используем только непрозрачные области)
        alpha = image[:, :, 3]
        mask = alpha > 128  # Берем только пиксели с альфа > 128
        
        # Создаем временное изображение для обнаружения лица
        temp_image = rgb_image.copy()
        # Для прозрачных областей устанавливаем нейтральный цвет
        temp_image[~mask] = [200, 200, 200]  # Светло-серый фон для прозрачных областей
    else:
        # Обычное изображение без прозрачности
        temp_image = image
    
    # Загружаем каскад Хаара для обнаружения лиц
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Преобразуем в оттенки серого для обнаружения лиц
    gray = cv2.cvtColor(temp_image, cv2.COLOR_BGR2GRAY)
    
    # Обнаружение лиц с разными параметрами для повышения точности
    faces = face_cascade.detectMultiScale(
        gray, 
        scaleFactor=1.1, 
        minNeighbors=5, 
        minSize=(30, 30)
    )
    
    if len(faces) == 0:
        # Пробуем с менее строгими параметрами
        faces = face_cascade.detectMultiScale(
            gray, 
            scaleFactor=1.2, 
            minNeighbors=3, 
            minSize=(20, 20)
        )
    
    if len(faces) == 0:
        return None
    
    # Берем наибольшее обнаруженное лицо
    if len(faces) > 1:
        areas = [w*h for (x, y, w, h) in faces]
        max_idx = areas.index(max(areas))
        x, y, w, h = faces[max_idx]
    else:
        x, y, w, h = faces[0]
    
    return {
        'x': x,
        'y': y,
        'width': w,
        'height': h,
        'center_x': x + w // 2,
        'center_y': y + h // 2
    }

# Функция для определения границ непрозрачного контента
def get_content_bounds(image):
    """Находит границы непрозрачного контента в PNG"""
    # Получаем размеры изображения
    h, w = image.shape[:2]
    
    # Проверяем наличие альфа-канала
    if len(image.shape) == 3 and image.shape[2] == 4:
        alpha = image[:, :, 3]
        # Находим все непрозрачные пиксели (с альфа > 10)
        non_transparent = alpha > 10
        
        # Если нет непрозрачных пикселей, возвращаем всё изображение
        if not np.any(non_transparent):
            return (0, 0, w, h)
        
        # Находим координаты непрозрачных пикселей
        rows = np.any(non_transparent, axis=1)
        cols = np.any(non_transparent, axis=0)
        
        # Находим границы непрозрачного контента
        y_min, y_max = np.where(rows)[0][[0, -1]]
        x_min, x_max = np.where(cols)[0][[0, -1]]
        
        # Добавляем отступ для безопасности
        padding = 10
        y_min = max(0, y_min - padding)
        y_max = min(h - 1, y_max + padding)
        x_min = max(0, x_min - padding)
        x_max = min(w - 1, x_max + padding)
        
        return (x_min, y_min, x_max - x_min + 1, y_max - y_min + 1)
    else:
        # Для изображений без прозрачности возвращаем всё изображение
        return (0, 0, w, h)

# Функция для нормализации PNG с настраиваемыми параметрами
def normalize_with_params(image, face_ratio=0.11, face_position=0.10, scale_factor=0.75, 
                          target_width=1000, target_height=1800, debug=False):
    """Нормализует PNG с настраиваемыми параметрами"""
    # Получаем размеры изображения
    h, w = image.shape[:2]
    
    # Пытаемся найти лицо
    face_data = detect_face(image)
    
    # Определяем границы контента
    x, y, content_width, content_height = get_content_bounds(image)
    
    # Создаем новое изображение с целевыми размерами и прозрачностью
    if len(image.shape) == 3 and image.shape[2] == 4:
        portrait = np.zeros((target_height, target_width, 4), dtype=np.uint8)
    else:
        portrait = np.zeros((target_height, target_width, 3), dtype=np.uint8)
    
    # Если нашли лицо, используем его для позиционирования
    if face_data:
        # Вычисляем масштаб с заданными параметрами
        face_height = face_data['height']
        scale = (target_height * face_ratio) / face_height * scale_factor
        
        # Определяем позицию лица в итоговом портрете
        target_face_y = int(target_height * face_position)
        
        # Вычисляем смещения для переноса всего изображения
        offset_x = target_width // 2 - int(face_data['center_x'] * scale)
        offset_y = target_face_y - int(face_data['y'] * scale)
    else:
        # Если лицо не найдено, масштабируем по размеру контента
        # Определяем масштаб, чтобы весь контент поместился
        content_ratio = content_width / content_height
        target_ratio = target_width / target_height
        
        if content_ratio > target_ratio:
            # Контент шире, масштабируем по ширине
            scale = target_width / content_width
        else:
            # Контент выше, масштабируем по высоте
            scale = target_height / content_height
        
        # Применяем дополнительный множитель масштаба
        scale *= scale_factor
        
        # Центрируем контент
        offset_x = (target_width - int(content_width * scale)) // 2
        offset_y = (target_height - int(content_height * scale)) // 2
    
    # Применяем масштаб к исходному изображению
    scaled_width = int(w * scale)
    scaled_height = int(h * scale)
    
    # Масштабируем только если изображение не слишком большое
    # (ограничиваем размер для экономии памяти)
    if scaled_width * scaled_height > 10000000:  # 10 мегапикселей
        # Для очень больших изображений сначала уменьшаем до разумного размера
        temp_scale = np.sqrt(10000000 / (w * h))
        temp_width = int(w * temp_scale)
        temp_height = int(h * temp_scale)
        temp_image = cv2.resize(image, (temp_width, temp_height), interpolation=cv2.INTER_AREA)
        
        # Затем масштабируем до нужного размера
        final_width = int(temp_width * (scale / temp_scale))
        final_height = int(temp_height * (scale / temp_scale))
        scaled_image = cv2.resize(temp_image, (final_width, final_height), interpolation=cv2.INTER_LANCZOS4)
        
        # Обновляем размеры
        scaled_width = final_width
        scaled_height = final_height
    else:
        # Для изображений разумного размера масштабируем напрямую
        scaled_image = cv2.resize(image, (scaled_width, scaled_height), interpolation=cv2.INTER_LANCZOS4)
    
    # Определяем область для вставки масштабированного изображения в портрет
    # с учетом смещений
    x_start = max(0, offset_x)
    y_start = max(0, offset_y)
    x_end = min(target_width, offset_x + scaled_width)
    y_end = min(target_height, offset_y + scaled_height)
    
    # Определяем область исходного изображения для копирования
    src_x_start = max(0, -offset_x)
    src_y_start = max(0, -offset_y)
    src_x_end = src_x_start + (x_end - x_start)
    src_y_end = src_y_start + (y_end - y_start)
    
    # Копируем часть масштабированного изображения в портрет с учетом прозрачности
    if len(scaled_image.shape) == 3 and scaled_image.shape[2] == 4:
        # С альфа-каналом
        scaled_section = scaled_image[src_y_start:src_y_end, src_x_start:src_x_end]
        
        # Проверяем, что размеры совпадают
        if scaled_section.shape[0] > 0 and scaled_section.shape[1] > 0:
            # Копируем RGB каналы и альфа-канал
            portrait[y_start:y_end, x_start:x_end, :3] = scaled_section[:, :, :3]
            portrait[y_start:y_end, x_start:x_end, 3] = scaled_section[:, :, 3]
    else:
        # Без альфа-канала (для полноты)
        scaled_section = scaled_image[src_y_start:src_y_end, src_x_start:src_x_end]
        
        # Проверяем, что размеры совпадают
        if scaled_section.shape[0] > 0 and scaled_section.shape[1] > 0:
            portrait[y_start:y_end, x_start:x_end] = scaled_section
    
    # Если включен режим отладки, отмечаем лицо на результате
    if debug and face_data:
        # Создаем копию для отрисовки
        debug_portrait = portrait.copy()
        
        # Вычисляем координаты лица в новом масштабе
        face_x = int(face_data['x'] * scale) + offset_x
        face_y = int(face_data['y'] * scale) + offset_y
        face_w = int(face_data['width'] * scale)
        face_h = int(face_data['height'] * scale)
        
        # Проверяем, что координаты в пределах изображения
        if (0 <= face_x < target_width and 0 <= face_y < target_height and
            0 <= face_x + face_w < target_width and 0 <= face_y + face_h < target_height):
            # Рисуем прямоугольник вокруг лица
            if len(debug_portrait.shape) == 3 and debug_portrait.shape[2] == 4:
                # Для изображений с альфа-каналом
                color = (0, 255, 0, 255)  # Зеленый, полностью непрозрачный
                thickness = 2
                cv2.rectangle(debug_portrait, (face_x, face_y), (face_x + face_w, face_y + face_h), color[:3], thickness)
                
                # Добавляем горизонтальную линию для визуализации положения лица
                y_line = int(target_height * face_position)
                cv2.line(debug_portrait, (0, y_line), (target_width, y_line), (255, 0, 0, 255), 1)
            else:
                # Для обычных изображений
                color = (0, 255, 0)  # Зеленый
                thickness = 2
                cv2.rectangle(debug_portrait, (face_x, face_y), (face_x + face_w, face_y + face_h), color, thickness)
                
                # Добавляем горизонтальную линию для визуализации положения лица
                y_line = int(target_height * face_position)
                cv2.line(debug_portrait, (0, y_line), (target_width, y_line), (255, 0, 0), 1)
        
        return debug_portrait
    
    return portrait

# Функция для сохранения изображения с прозрачностью
def save_image_with_transparency(image, output_path):
    # Для PNG сохраняем с прозрачностью
    if output_path.lower().endswith('.png'):
        if len(image.shape) == 3 and image.shape[2] == 4:
            # Если это PNG с альфа-каналом, сохраняем как есть
            cv2.imwrite(output_path, image)
        else:
            # Если нет альфа-канала, сохраняем как обычное изображение
            cv2.imwrite(output_path, image)
    else:
        # Для других форматов удаляем альфа-канал
        if len(image.shape) == 3 and image.shape[2] == 4:
            # Если есть альфа-канал, берем только RGB часть
            cv2.imwrite(output_path, image[:, :, 0:3])
        else:
            # Если нет альфа-канала, сохраняем как обычное изображение
            cv2.imwrite(output_path, image)
    return output_path

# Функция для отображения изображений в ноутбуке
def display_images(images, titles=None, figsize=(15, 10)):
    """Отображает список изображений в Jupyter Notebook"""
    n = len(images)
    if titles is None:
        titles = ['Image {}'.format(i) for i in range(1, n + 1)]
    fig, axs = plt.subplots(1, n, figsize=figsize)
    if n == 1:
        axs = [axs]
    for i in range(n):
        img = images[i]
        if img is None:
            continue
            
        if len(img.shape) == 2:  # Если изображение в градациях серого
            axs[i].imshow(img, cmap='gray')
        else:  # Если изображение цветное
            if img.shape[2] == 3:  # Если RGB
                axs[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            elif img.shape[2] == 4:  # Если RGBA
                # Преобразуем BGRA в RGBA для правильного отображения
                rgb = cv2.cvtColor(img[:, :, 0:3], cv2.COLOR_BGR2RGB)
                rgba = np.dstack((rgb, img[:, :, 3]))
                axs[i].imshow(rgba)
        axs[i].set_title(titles[i])
        axs[i].axis('off')
    plt.tight_layout()
    return fig

# Класс для интерактивного виджета настройки изображений
class ImageNormalizer:
    def __init__(self):
        self.images = get_image_list()
        self.current_image = None
        self.current_index = 0
        self.original_image = None
        self.processed_image = None
        self.saved_params = {}  # Словарь для сохранения параметров для каждого изображения
        
        # Создаем виджеты
        self.image_dropdown = Dropdown(
            options=self.images,
            description='Изображение:',
            style={'description_width': 'initial'},
            layout={'width': '500px'}
        )
        
        self.face_ratio_slider = FloatSlider(
            value=0.11,
            min=0.05,
            max=0.25,
            step=0.01,
            description='Размер лица:',
            style={'description_width': 'initial'},
            layout={'width': '500px'}
        )
        
        self.face_position_slider = FloatSlider(
            value=0.10,
            min=0.05,
            max=0.25,
            step=0.01,
            description='Положение лица:',
            style={'description_width': 'initial'},
            layout={'width': '500px'}
        )
        
        self.scale_factor_slider = FloatSlider(
            value=0.75,
            min=0.4,
            max=1.2,
            step=0.05,
            description='Множитель масштаба:',
            style={'description_width': 'initial'},
            layout={'width': '500px'}
        )
        
        self.width_slider = IntSlider(
            value=1000,
            min=600,
            max=2000,
            step=50,
            description='Ширина:',
            style={'description_width': 'initial'},
            layout={'width': '500px'}
        )
        
        self.height_slider = IntSlider(
            value=1800,
            min=900,
            max=3000,
            step=50,
            description='Высота:',
            style={'description_width': 'initial'},
            layout={'width': '500px'}
        )
        
        self.debug_checkbox = Checkbox(
            value=True,
            description='Показать маркеры отладки',
            style={'description_width': 'initial'}
        )
        
        self.process_button = Button(
            description='Обработать',
            button_style='info',
            tooltip='Применить текущие настройки',
            icon='refresh'
        )
        
        self.save_button = Button(
            description='Сохранить',
            button_style='success',
            tooltip='Сохранить обработанное изображение',
            icon='save'
        )
        
        self.save_params_button = Button(
            description='Запомнить параметры',
            button_style='warning',
            tooltip='Запомнить текущие параметры для этого изображения',
            icon='bookmark'
        )
        
        self.batch_process_button = Button(
            description='Пакетная обработка всех изображений',
            button_style='danger',
            tooltip='Обработать все изображения с их сохраненными параметрами',
            icon='play'
        )
        
        self.output = Output()
        
        # Настройка обработчиков событий
        self.image_dropdown.observe(self.on_image_change, names='value')
        self.process_button.on_click(self.on_process_click)
        self.save_button.on_click(self.on_save_click)
        self.save_params_button.on_click(self.on_save_params_click)
        self.batch_process_button.on_click(self.on_batch_process_click)
        
        # Первое изображение
        if self.images:
            self.current_image = self.images[0]
            self.load_current_image()
    
    def load_current_image(self):
        """Загружает текущее выбранное изображение"""
        if not self.current_image:
            return
            
        image_path = os.path.join(SOURCE_DIR, self.current_image)
        self.original_image = load_image_with_alpha(image_path)
        
        if self.original_image is None:
            print(f"Не удалось загрузить изображение: {image_path}")
            return
            
        # Если для текущего изображения есть сохраненные параметры, загружаем их
        if self.current_image in self.saved_params:
            params = self.saved_params[self.current_image]
            self.face_ratio_slider.value = params['face_ratio']
            self.face_position_slider.value = params['face_position']
            self.scale_factor_slider.value = params['scale_factor']
            self.width_slider.value = params['width']
            self.height_slider.value = params['height']
            
        # Обрабатываем изображение с текущими параметрами
        self.process_current_image()
    
    def process_current_image(self):
        """Обрабатывает текущее изображение с заданными параметрами"""
        if self.original_image is None:
            return
            
        self.processed_image = normalize_with_params(
            self.original_image,
            face_ratio=self.face_ratio_slider.value,
            face_position=self.face_position_slider.value,
            scale_factor=self.scale_factor_slider.value,
            target_width=self.width_slider.value,
            target_height=self.height_slider.value,
            debug=self.debug_checkbox.value
        )
        
        # Отображаем результат
        self.display_results()
    
    def display_results(self):
        """Отображает исходное и обработанное изображения"""
        with self.output:
            clear_output(wait=True)
            if self.original_image is not None and self.processed_image is not None:
                fig = display_images(
                    [self.original_image, self.processed_image],
                    ['Исходное', 'Обработанное'],
                    figsize=(15, 8)
                )
                plt.show()
    
    def on_image_change(self, change):
        """Обработчик изменения выбранного изображения"""
        if change['type'] == 'change' and change['name'] == 'value':
            self.current_image = change['new']
            self.load_current_image()
    
    def on_process_click(self, b):
        """Обработчик нажатия кнопки 'Обработать'"""
        self.process_current_image()
    
    def on_save_click(self, b):
        """Обработчик нажатия кнопки 'Сохранить'"""
        if self.processed_image is None or self.current_image is None:
            print("Нет обработанного изображения для сохранения")
            return
            
        output_path = os.path.join(TARGET_DIR, self.current_image)
        
        # Сохраняем без маркеров отладки
        clean_image = normalize_with_params(
            self.original_image,
            face_ratio=self.face_ratio_slider.value,
            face_position=self.face_position_slider.value,
            scale_factor=self.scale_factor_slider.value,
            target_width=self.width_slider.value,
            target_height=self.height_slider.value,
            debug=False
        )
        
        save_image_with_transparency(clean_image, output_path)
        print(f"Сохранено: {output_path}")
    
    def on_save_params_click(self, b):
        """Обработчик нажатия кнопки 'Запомнить параметры'"""
        if self.current_image is None:
            return
            
        # Сохраняем текущие параметры для этого изображения
        self.saved_params[self.current_image] = {
            'face_ratio': self.face_ratio_slider.value,
            'face_position': self.face_position_slider.value,
            'scale_factor': self.scale_factor_slider.value,
            'width': self.width_slider.value,
            'height': self.height_slider.value
        }
        
        print(f"Параметры для {self.current_image} сохранены")
    
    def on_batch_process_click(self, b):
        """Обработчик нажатия кнопки 'Пакетная обработка'"""
        processed_count = 0
        
        for img_name in self.images:
            if img_name in self.saved_params:
                # Загружаем изображение
                img_path = os.path.join(SOURCE_DIR, img_name)
                image = load_image_with_alpha(img_path)
                
                if image is None:
                    print(f"Не удалось загрузить: {img_name}")
                    continue
                    
                # Получаем сохраненные параметры
                params = self.saved_params[img_name]
                
                # Обрабатываем с этими параметрами
                processed = normalize_with_params(
                    image,
                    face_ratio=params['face_ratio'],
                    face_position=params['face_position'],
                    scale_factor=params['scale_factor'],
                    target_width=params['width'],
                    target_height=params['height'],
                    debug=False
                )
                
                # Сохраняем результат
                output_path = os.path.join(TARGET_DIR, img_name)
                save_image_with_transparency(processed, output_path)
                
                processed_count += 1
                print(f"Обработано и сохранено: {img_name}")
        
        print(f"\nВсего обработано: {processed_count} из {len(self.images)} изображений")
        print(f"Результаты сохранены в папку: {TARGET_DIR}")
    
    def create_ui(self):
        """Создает и отображает пользовательский интерфейс"""
        # Компоновка виджетов
        image_selector = VBox([self.image_dropdown])
        
        params_box = VBox([
            self.face_ratio_slider,
            self.face_position_slider,
            self.scale_factor_slider,
            self.width_slider,
            self.height_slider,
            self.debug_checkbox
        ])
        
        buttons_box = HBox([
            self.process_button,
            self.save_button,
            self.save_params_button
        ])
        
        batch_box = VBox([self.batch_process_button])
        
        # Главный контейнер
        main_ui = VBox([
            image_selector,
            params_box,
            buttons_box,
            batch_box,
            self.output
        ])
        
        display(main_ui)

# Создаем и отображаем интерфейс
def create_normalizer_ui():
    normalizer = ImageNormalizer()
    normalizer.create_ui()
    return normalizer

# Вызываем функцию создания интерфейса
create_normalizer_ui()

### C гранями

In [4]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox
from ipywidgets import FloatSlider, IntSlider, Dropdown, Output, Text, Checkbox, Tab
from IPython.display import display, clear_output
import glob

# Папки для обработки
SOURCE_DIR = 'source'  # Папка с исходными изображениями
TARGET_DIR = 'final'   # Папка для сохранения результатов

# Создаем целевую папку, если она не существует
if not os.path.exists(TARGET_DIR):
    os.makedirs(TARGET_DIR)
    print(f"Создана папка: {TARGET_DIR}")

# Получаем список изображений в исходной папке
def get_image_list():
    images = []
    for ext in ['png', 'jpg', 'jpeg']:
        images.extend(glob.glob(os.path.join(SOURCE_DIR, f'*.{ext}')))
    return sorted([os.path.basename(img) for img in images])

# Функция для загрузки PNG с сохранением альфа-канала
def load_image_with_alpha(image_path):
    # Проверяем, является ли файл PNG
    if image_path.lower().endswith('.png'):
        # Загружаем с альфа-каналом
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        
        if image is None:
            return None
            
        # Проверяем наличие альфа-канала
        if len(image.shape) == 3 and image.shape[2] == 4:
            return image
    
    # Для других форматов или PNG без прозрачности загружаем обычным способом
    return cv2.imread(image_path)

# Функция для обнаружения лица на изображении
def detect_face(image):
    """Определяет положение и размер лица на изображении используя OpenCV"""
    # Проверяем наличие альфа-канала
    if len(image.shape) == 3 and image.shape[2] == 4:
        # Создаем копию только RGB каналов
        rgb_image = image[:, :, :3].copy()
        # Создаем маску прозрачности (используем только непрозрачные области)
        alpha = image[:, :, 3]
        mask = alpha > 128  # Берем только пиксели с альфа > 128
        
        # Создаем временное изображение для обнаружения лица
        temp_image = rgb_image.copy()
        # Для прозрачных областей устанавливаем нейтральный цвет
        temp_image[~mask] = [200, 200, 200]  # Светло-серый фон для прозрачных областей
    else:
        # Обычное изображение без прозрачности
        temp_image = image
    
    # Загружаем каскад Хаара для обнаружения лиц
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Преобразуем в оттенки серого для обнаружения лиц
    gray = cv2.cvtColor(temp_image, cv2.COLOR_BGR2GRAY)
    
    # Обнаружение лиц с разными параметрами для повышения точности
    faces = face_cascade.detectMultiScale(
        gray, 
        scaleFactor=1.1, 
        minNeighbors=5, 
        minSize=(30, 30)
    )
    
    if len(faces) == 0:
        # Пробуем с менее строгими параметрами
        faces = face_cascade.detectMultiScale(
            gray, 
            scaleFactor=1.2, 
            minNeighbors=3, 
            minSize=(20, 20)
        )
    
    if len(faces) == 0:
        return None
    
    # Берем наибольшее обнаруженное лицо
    if len(faces) > 1:
        areas = [w*h for (x, y, w, h) in faces]
        max_idx = areas.index(max(areas))
        x, y, w, h = faces[max_idx]
    else:
        x, y, w, h = faces[0]
    
    return {
        'x': x,
        'y': y,
        'width': w,
        'height': h,
        'center_x': x + w // 2,
        'center_y': y + h // 2
    }

# Функция для определения границ непрозрачного контента
def get_content_bounds(image):
    """Находит границы непрозрачного контента в PNG"""
    # Получаем размеры изображения
    h, w = image.shape[:2]
    
    # Проверяем наличие альфа-канала
    if len(image.shape) == 3 and image.shape[2] == 4:
        alpha = image[:, :, 3]
        # Находим все непрозрачные пиксели (с альфа > 10)
        non_transparent = alpha > 10
        
        # Если нет непрозрачных пикселей, возвращаем всё изображение
        if not np.any(non_transparent):
            return (0, 0, w, h)
        
        # Находим координаты непрозрачных пикселей
        rows = np.any(non_transparent, axis=1)
        cols = np.any(non_transparent, axis=0)
        
        # Находим границы непрозрачного контента
        y_min, y_max = np.where(rows)[0][[0, -1]]
        x_min, x_max = np.where(cols)[0][[0, -1]]
        
        # Добавляем отступ для безопасности
        padding = 10
        y_min = max(0, y_min - padding)
        y_max = min(h - 1, y_max + padding)
        x_min = max(0, x_min - padding)
        x_max = min(w - 1, x_max + padding)
        
        return (x_min, y_min, x_max - x_min + 1, y_max - y_min + 1)
    else:
        # Для изображений без прозрачности возвращаем всё изображение
        return (0, 0, w, h)

# Функция для нормализации PNG с настраиваемыми параметрами и отступами
def normalize_with_params(image, face_ratio=0.11, face_position=0.10, scale_factor=0.75, 
                          target_width=530, target_height=1000, debug=False,
                          offset_left=0, offset_right=0, offset_top=0, offset_bottom=0):
    """Нормализует PNG с настраиваемыми параметрами и отступами"""
    # Получаем размеры изображения
    h, w = image.shape[:2]
    
    # Пытаемся найти лицо
    face_data = detect_face(image)
    
    # Определяем границы контента
    x, y, content_width, content_height = get_content_bounds(image)
    
    # Создаем новое изображение с целевыми размерами и прозрачностью
    if len(image.shape) == 3 and image.shape[2] == 4:
        portrait = np.zeros((target_height, target_width, 4), dtype=np.uint8)
    else:
        portrait = np.zeros((target_height, target_width, 3), dtype=np.uint8)
    
    # Если нашли лицо, используем его для позиционирования
    if face_data:
        # Вычисляем масштаб с заданными параметрами
        face_height = face_data['height']
        scale = (target_height * face_ratio) / face_height * scale_factor
        
        # Определяем позицию лица в итоговом портрете
        target_face_y = int(target_height * face_position)
        
        # Вычисляем смещения для переноса всего изображения с учетом дополнительных отступов
        offset_x = target_width // 2 - int(face_data['center_x'] * scale) + offset_left - offset_right
        offset_y = target_face_y - int(face_data['y'] * scale) + offset_top - offset_bottom
    else:
        # Если лицо не найдено, масштабируем по размеру контента
        # Определяем масштаб, чтобы весь контент поместился
        content_ratio = content_width / content_height
        target_ratio = target_width / target_height
        
        if content_ratio > target_ratio:
            # Контент шире, масштабируем по ширине
            scale = target_width / content_width
        else:
            # Контент выше, масштабируем по высоте
            scale = target_height / content_height
        
        # Применяем дополнительный множитель масштаба
        scale *= scale_factor
        
        # Центрируем контент с учетом отступов
        offset_x = (target_width - int(content_width * scale)) // 2 + offset_left - offset_right
        offset_y = (target_height - int(content_height * scale)) // 2 + offset_top - offset_bottom
    
    # Применяем масштаб к исходному изображению
    scaled_width = int(w * scale)
    scaled_height = int(h * scale)
    
    # Масштабируем только если изображение не слишком большое
    # (ограничиваем размер для экономии памяти)
    if scaled_width * scaled_height > 10000000:  # 10 мегапикселей
        # Для очень больших изображений сначала уменьшаем до разумного размера
        temp_scale = np.sqrt(10000000 / (w * h))
        temp_width = int(w * temp_scale)
        temp_height = int(h * temp_scale)
        temp_image = cv2.resize(image, (temp_width, temp_height), interpolation=cv2.INTER_AREA)
        
        # Затем масштабируем до нужного размера
        final_width = int(temp_width * (scale / temp_scale))
        final_height = int(temp_height * (scale / temp_scale))
        scaled_image = cv2.resize(temp_image, (final_width, final_height), interpolation=cv2.INTER_LANCZOS4)
        
        # Обновляем размеры
        scaled_width = final_width
        scaled_height = final_height
    else:
        # Для изображений разумного размера масштабируем напрямую
        scaled_image = cv2.resize(image, (scaled_width, scaled_height), interpolation=cv2.INTER_LANCZOS4)
    
    # Определяем область для вставки масштабированного изображения в портрет
    # с учетом смещений
    x_start = max(0, offset_x)
    y_start = max(0, offset_y)
    x_end = min(target_width, offset_x + scaled_width)
    y_end = min(target_height, offset_y + scaled_height)
    
    # Проверяем, есть ли пересечение
    if x_start >= target_width or y_start >= target_height or x_end <= 0 or y_end <= 0:
        # Нет пересечения, возвращаем пустое изображение
        return portrait
    
    # Определяем область исходного изображения для копирования
    src_x_start = max(0, -offset_x)
    src_y_start = max(0, -offset_y)
    src_x_end = src_x_start + (x_end - x_start)
    src_y_end = src_y_start + (y_end - y_start)
    
    # Копируем часть масштабированного изображения в портрет с учетом прозрачности
    if len(scaled_image.shape) == 3 and scaled_image.shape[2] == 4:
        # С альфа-каналом
        scaled_section = scaled_image[src_y_start:src_y_end, src_x_start:src_x_end]
        
        # Проверяем, что размеры совпадают
        if scaled_section.shape[0] > 0 and scaled_section.shape[1] > 0:
            # Копируем RGB каналы и альфа-канал
            portrait[y_start:y_end, x_start:x_end, :3] = scaled_section[:, :, :3]
            portrait[y_start:y_end, x_start:x_end, 3] = scaled_section[:, :, 3]
    else:
        # Без альфа-канала (для полноты)
        scaled_section = scaled_image[src_y_start:src_y_end, src_x_start:src_x_end]
        
        # Проверяем, что размеры совпадают
        if scaled_section.shape[0] > 0 and scaled_section.shape[1] > 0:
            portrait[y_start:y_end, x_start:x_end] = scaled_section
    
    # Если включен режим отладки, отмечаем лицо на результате
    if debug and face_data:
        # Создаем копию для отрисовки
        debug_portrait = portrait.copy()
        
        # Вычисляем координаты лица в новом масштабе
        face_x = int(face_data['x'] * scale) + offset_x
        face_y = int(face_data['y'] * scale) + offset_y
        face_w = int(face_data['width'] * scale)
        face_h = int(face_data['height'] * scale)
        
        # Проверяем, что координаты в пределах изображения
        if (0 <= face_x < target_width and 0 <= face_y < target_height and
            0 <= face_x + face_w < target_width and 0 <= face_y + face_h < target_height):
            # Рисуем прямоугольник вокруг лица
            if len(debug_portrait.shape) == 3 and debug_portrait.shape[2] == 4:
                # Для изображений с альфа-каналом
                color = (0, 255, 0, 255)  # Зеленый, полностью непрозрачный
                thickness = 2
                cv2.rectangle(debug_portrait, (face_x, face_y), (face_x + face_w, face_y + face_h), color[:3], thickness)
                
                # Добавляем горизонтальную линию для визуализации положения лица
                y_line = int(target_height * face_position)
                cv2.line(debug_portrait, (0, y_line), (target_width, y_line), (255, 0, 0, 255), 1)
                
                # Добавляем линии для визуализации отступов
                # Вертикальные линии для отступов слева и справа
                cv2.line(debug_portrait, (offset_left, 0), (offset_left, target_height), (255, 255, 0, 255), 1)
                right_line = target_width - offset_right
                cv2.line(debug_portrait, (right_line, 0), (right_line, target_height), (255, 255, 0, 255), 1)
                
                # Горизонтальные линии для отступов сверху и снизу
                cv2.line(debug_portrait, (0, offset_top), (target_width, offset_top), (0, 255, 255, 255), 1)
                bottom_line = target_height - offset_bottom
                cv2.line(debug_portrait, (0, bottom_line), (target_width, bottom_line), (0, 255, 255, 255), 1)
            else:
                # Для обычных изображений
                color = (0, 255, 0)  # Зеленый
                thickness = 2
                cv2.rectangle(debug_portrait, (face_x, face_y), (face_x + face_w, face_y + face_h), color, thickness)
                
                # Добавляем горизонтальную линию для визуализации положения лица
                y_line = int(target_height * face_position)
                cv2.line(debug_portrait, (0, y_line), (target_width, y_line), (255, 0, 0), 1)
                
                # Добавляем линии для визуализации отступов
                # Вертикальные линии для отступов слева и справа
                cv2.line(debug_portrait, (offset_left, 0), (offset_left, target_height), (255, 255, 0), 1)
                right_line = target_width - offset_right
                cv2.line(debug_portrait, (right_line, 0), (right_line, target_height), (255, 255, 0), 1)
                
                # Горизонтальные линии для отступов сверху и снизу
                cv2.line(debug_portrait, (0, offset_top), (target_width, offset_top), (0, 255, 255), 1)
                bottom_line = target_height - offset_bottom
                cv2.line(debug_portrait, (0, bottom_line), (target_width, bottom_line), (0, 255, 255), 1)
        
        return debug_portrait
    
    return portrait

# Функция для сохранения изображения с прозрачностью
def save_image_with_transparency(image, output_path):
    # Для PNG сохраняем с прозрачностью
    if output_path.lower().endswith('.png'):
        if len(image.shape) == 3 and image.shape[2] == 4:
            # Если это PNG с альфа-каналом, сохраняем как есть
            cv2.imwrite(output_path, image)
        else:
            # Если нет альфа-канала, сохраняем как обычное изображение
            cv2.imwrite(output_path, image)
    else:
        # Для других форматов удаляем альфа-канал
        if len(image.shape) == 3 and image.shape[2] == 4:
            # Если есть альфа-канал, берем только RGB часть
            cv2.imwrite(output_path, image[:, :, 0:3])
        else:
            # Если нет альфа-канала, сохраняем как обычное изображение
            cv2.imwrite(output_path, image)
    return output_path

# Функция для отображения изображений в ноутбуке
def display_images(images, titles=None, figsize=(15, 10)):
    """Отображает список изображений в Jupyter Notebook"""
    n = len(images)
    if titles is None:
        titles = ['Image {}'.format(i) for i in range(1, n + 1)]
    fig, axs = plt.subplots(1, n, figsize=figsize)
    if n == 1:
        axs = [axs]
    for i in range(n):
        img = images[i]
        if img is None:
            continue
            
        if len(img.shape) == 2:  # Если изображение в градациях серого
            axs[i].imshow(img, cmap='gray')
        else:  # Если изображение цветное
            if img.shape[2] == 3:  # Если RGB
                axs[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            elif img.shape[2] == 4:  # Если RGBA
                # Преобразуем BGRA в RGBA для правильного отображения
                rgb = cv2.cvtColor(img[:, :, 0:3], cv2.COLOR_BGR2RGB)
                rgba = np.dstack((rgb, img[:, :, 3]))
                axs[i].imshow(rgba)
        axs[i].set_title(titles[i])
        axs[i].axis('off')
    plt.tight_layout()
    return fig

# Класс для интерактивного виджета настройки изображений
class ImageNormalizer:
    def __init__(self):
        self.images = get_image_list()
        self.current_image = None
        self.current_index = 0
        self.original_image = None
        self.processed_image = None
        self.saved_params = {}  # Словарь для сохранения параметров для каждого изображения
        
        # Создаем виджеты
        self.image_dropdown = Dropdown(
            options=self.images,
            description='Изображение:',
            style={'description_width': 'initial'},
            layout={'width': '500px'}
        )
        
        # Основные параметры с расширенными диапазонами
        self.face_ratio_slider = FloatSlider(
            value=0.11,
            min=0.03,
            max=0.30,
            step=0.01,
            description='Размер лица:',
            style={'description_width': 'initial'},
            layout={'width': '500px'}
        )
        
        self.face_position_slider = FloatSlider(
            value=0.10,
            min=0.03,
            max=0.30,
            step=0.01,
            description='Положение лица:',
            style={'description_width': 'initial'},
            layout={'width': '500px'}
        )
        
        self.scale_factor_slider = FloatSlider(
            value=0.75,
            min=0.2,
            max=1.5,
            step=0.05,
            description='Множитель масштаба:',
            style={'description_width': 'initial'},
            layout={'width': '500px'}
        )
        
        # Размеры изображения (по умолчанию 530x1000)
        self.width_slider = IntSlider(
            value=530,
            min=300,
            max=2000,
            step=10,
            description='Ширина:',
            style={'description_width': 'initial'},
            layout={'width': '500px'}
        )
        
        self.height_slider = IntSlider(
            value=1000,
            min=500,
            max=3000,
            step=10,
            description='Высота:',
            style={'description_width': 'initial'},
            layout={'width': '500px'}
        )
        
        # Новые слайдеры для отступов
        self.offset_left_slider = IntSlider(
            value=0,
            min=-200,
            max=200,
            step=5,
            description='Отступ слева:',
            style={'description_width': 'initial'},
            layout={'width': '500px'}
        )
        
        self.offset_right_slider = IntSlider(
            value=0,
            min=-200,
            max=200,
            step=5,
            description='Отступ справа:',
            style={'description_width': 'initial'},
            layout={'width': '500px'}
        )
        
        self.offset_top_slider = IntSlider(
            value=0,
            min=-200,
            max=200,
            step=5,
            description='Отступ сверху:',
            style={'description_width': 'initial'},
            layout={'width': '500px'}
        )
        
        self.offset_bottom_slider = IntSlider(
            value=0,
            min=-200,
            max=200,
            step=5,
            description='Отступ снизу:',
            style={'description_width': 'initial'},
            layout={'width': '500px'}
        )
        
        self.debug_checkbox = Checkbox(
            value=True,
            description='Показать маркеры отладки',
            style={'description_width': 'initial'}
        )
        
        # Кнопки
        self.process_button = Button(
            description='Обработать',
            button_style='info',
            tooltip='Применить текущие настройки',
            icon='refresh'
        )
        
        self.save_button = Button(
            description='Сохранить',
            button_style='success',
            tooltip='Сохранить обработанное изображение',
            icon='save'
        )
        
        self.save_params_button = Button(
            description='Запомнить параметры',
            button_style='warning',
            tooltip='Запомнить текущие параметры для этого изображения',
            icon='bookmark'
        )
        
        self.batch_process_button = Button(
            description='Пакетная обработка всех изображений',
            button_style='danger',
            tooltip='Обработать все изображения с их сохраненными параметрами',
            icon='play'
        )
        
        # Кнопки перехода между изображениями
        self.prev_button = Button(
            description='← Предыдущее',
            tooltip='Перейти к предыдущему изображению',
            button_style='',
            layout={'width': '150px'}
        )
        
        self.next_button = Button(
            description='Следующее →',
            tooltip='Перейти к следующему изображению',
            button_style='',
            layout={'width': '150px'}
        )
        
        self.reset_offsets_button = Button(
            description='Сбросить отступы',
            tooltip='Сбросить все отступы на 0',
            button_style='',
            layout={'width': '150px'}
        )
        
        self.output = Output()
        
        # Настройка обработчиков событий
        self.image_dropdown.observe(self.on_image_change, names='value')
        self.process_button.on_click(self.on_process_click)
        self.save_button.on_click(self.on_save_click)
        self.save_params_button.on_click(self.on_save_params_click)
        self.batch_process_button.on_click(self.on_batch_process_click)
        self.prev_button.on_click(self.on_prev_click)
        self.next_button.on_click(self.on_next_click)
        self.reset_offsets_button.on_click(self.on_reset_offsets_click)
        
        # Первое изображение
        if self.images:
            self.current_image = self.images[0]
            self.load_current_image()
    
    def load_current_image(self):
        """Загружает текущее выбранное изображение"""
        if not self.current_image:
            return
            
        image_path = os.path.join(SOURCE_DIR, self.current_image)
        self.original_image = load_image_with_alpha(image_path)
        
        if self.original_image is None:
            print(f"Не удалось загрузить изображение: {image_path}")
            return
            
        # Если для текущего изображения есть сохраненные параметры, загружаем их
        if self.current_image in self.saved_params:
            params = self.saved_params[self.current_image]
            self.face_ratio_slider.value = params.get('face_ratio', 0.11)
            self.face_position_slider.value = params.get('face_position', 0.10)
            self.scale_factor_slider.value = params.get('scale_factor', 0.75)
            self.width_slider.value = params.get('width', 530)
            self.height_slider.value = params.get('height', 1000)
            # Загружаем отступы, если они есть
            self.offset_left_slider.value = params.get('offset_left', 0)
            self.offset_right_slider.value = params.get('offset_right', 0)
            self.offset_top_slider.value = params.get('offset_top', 0)
            self.offset_bottom_slider.value = params.get('offset_bottom', 0)
            
        # Обрабатываем изображение с текущими параметрами
        self.process_current_image()
    
    def process_current_image(self):
        """Обрабатывает текущее изображение с заданными параметрами"""
        if self.original_image is None:
            return
            
        self.processed_image = normalize_with_params(
            self.original_image,
            face_ratio=self.face_ratio_slider.value,
            face_position=self.face_position_slider.value,
            scale_factor=self.scale_factor_slider.value,
            target_width=self.width_slider.value,
            target_height=self.height_slider.value,
            debug=self.debug_checkbox.value,
            offset_left=self.offset_left_slider.value,
            offset_right=self.offset_right_slider.value,
            offset_top=self.offset_top_slider.value,
            offset_bottom=self.offset_bottom_slider.value
        )
        
        # Отображаем результат
        self.display_results()
    
    def display_results(self):
        """Отображает исходное и обработанное изображения"""
        with self.output:
            clear_output(wait=True)
            if self.original_image is not None and self.processed_image is not None:
                fig = display_images(
                    [self.original_image, self.processed_image],
                    ['Исходное', 'Обработанное'],
                    figsize=(15, 8)
                )
                plt.show()
                
                # Выводим информацию о текущих параметрах
                print(f"Изображение: {self.current_image} ({self.current_index + 1}/{len(self.images)})")
                print(f"Размер лица: {self.face_ratio_slider.value:.2f}, Положение: {self.face_position_slider.value:.2f}, Масштаб: {self.scale_factor_slider.value:.2f}")
                print(f"Размеры: {self.width_slider.value}x{self.height_slider.value}")
                print(f"Отступы: Л:{self.offset_left_slider.value} П:{self.offset_right_slider.value} В:{self.offset_top_slider.value} Н:{self.offset_bottom_slider.value}")
    
    def on_image_change(self, change):
        """Обработчик изменения выбранного изображения"""
        if change['type'] == 'change' and change['name'] == 'value':
            self.current_image = change['new']
            self.current_index = self.images.index(self.current_image)
            self.load_current_image()
    
    def on_prev_click(self, b):
        """Обработчик нажатия кнопки 'Предыдущее'"""
        if not self.images:
            return
            
        self.current_index = (self.current_index - 1) % len(self.images)
        self.current_image = self.images[self.current_index]
        self.image_dropdown.value = self.current_image  # Обновляем выпадающий список
    
    def on_next_click(self, b):
        """Обработчик нажатия кнопки 'Следующее'"""
        if not self.images:
            return
            
        self.current_index = (self.current_index + 1) % len(self.images)
        self.current_image = self.images[self.current_index]
        self.image_dropdown.value = self.current_image  # Обновляем выпадающий список
    
    def on_reset_offsets_click(self, b):
        """Сбрасывает все отступы на 0"""
        self.offset_left_slider.value = 0
        self.offset_right_slider.value = 0
        self.offset_top_slider.value = 0
        self.offset_bottom_slider.value = 0
        self.process_current_image()
    
    def on_process_click(self, b):
        """Обработчик нажатия кнопки 'Обработать'"""
        self.process_current_image()
    
    def on_save_click(self, b):
        """Обработчик нажатия кнопки 'Сохранить'"""
        if self.processed_image is None or self.current_image is None:
            print("Нет обработанного изображения для сохранения")
            return
            
        output_path = os.path.join(TARGET_DIR, self.current_image)
        
        # Сохраняем без маркеров отладки
        clean_image = normalize_with_params(
            self.original_image,
            face_ratio=self.face_ratio_slider.value,
            face_position=self.face_position_slider.value,
            scale_factor=self.scale_factor_slider.value,
            target_width=self.width_slider.value,
            target_height=self.height_slider.value,
            debug=False,
            offset_left=self.offset_left_slider.value,
            offset_right=self.offset_right_slider.value,
            offset_top=self.offset_top_slider.value,
            offset_bottom=self.offset_bottom_slider.value
        )
        
        save_image_with_transparency(clean_image, output_path)
        print(f"Сохранено: {output_path}")
    
    def on_save_params_click(self, b):
        """Обработчик нажатия кнопки 'Запомнить параметры'"""
        if self.current_image is None:
            return
            
        # Сохраняем текущие параметры для этого изображения
        self.saved_params[self.current_image] = {
            'face_ratio': self.face_ratio_slider.value,
            'face_position': self.face_position_slider.value,
            'scale_factor': self.scale_factor_slider.value,
            'width': self.width_slider.value,
            'height': self.height_slider.value,
            'offset_left': self.offset_left_slider.value,
            'offset_right': self.offset_right_slider.value,
            'offset_top': self.offset_top_slider.value,
            'offset_bottom': self.offset_bottom_slider.value
        }
        
        print(f"✓ Параметры для {self.current_image} сохранены")
    
    def on_batch_process_click(self, b):
        """Обработчик нажатия кнопки 'Пакетная обработка'"""
        processed_count = 0
        
        for img_name in self.images:
            if img_name in self.saved_params:
                # Загружаем изображение
                img_path = os.path.join(SOURCE_DIR, img_name)
                image = load_image_with_alpha(img_path)
                
                if image is None:
                    print(f"Не удалось загрузить: {img_name}")
                    continue
                    
                # Получаем сохраненные параметры
                params = self.saved_params[img_name]
                
                # Обрабатываем с этими параметрами
                processed = normalize_with_params(
                    image,
                    face_ratio=params.get('face_ratio', 0.11),
                    face_position=params.get('face_position', 0.10),
                    scale_factor=params.get('scale_factor', 0.75),
                    target_width=params.get('width', 530),
                    target_height=params.get('height', 1000),
                    debug=False,
                    offset_left=params.get('offset_left', 0),
                    offset_right=params.get('offset_right', 0),
                    offset_top=params.get('offset_top', 0),
                    offset_bottom=params.get('offset_bottom', 0)
                )
                
                # Сохраняем результат
                output_path = os.path.join(TARGET_DIR, img_name)
                save_image_with_transparency(processed, output_path)
                
                processed_count += 1
                print(f"✓ Обработано: {img_name}")
            else:
                print(f"Пропущено: {img_name} (нет сохраненных параметров)")
        
        print(f"\n--- Результаты обработки ---")
        print(f"Всего обработано: {processed_count} из {len(self.images)} изображений")
        print(f"Результаты сохранены в папку: {TARGET_DIR}")
    
    def create_ui(self):
        """Создает и отображает пользовательский интерфейс"""
        # Навигация между изображениями
        navigation_box = HBox([self.prev_button, self.next_button])
        
        # Компоновка виджетов
        image_selector = VBox([
            HBox([self.image_dropdown]), 
            navigation_box
        ])
        
        # Вкладки для разных групп параметров
        tab_main = VBox([
            self.face_ratio_slider,
            self.face_position_slider,
            self.scale_factor_slider,
            self.width_slider,
            self.height_slider,
            self.debug_checkbox
        ])
        
        tab_offsets = VBox([
            self.offset_left_slider,
            self.offset_right_slider,
            self.offset_top_slider,
            self.offset_bottom_slider,
            self.reset_offsets_button
        ])
        
        # Создаем вкладки
        tabs = Tab()
        tabs.children = [tab_main, tab_offsets]
        tabs.set_title(0, 'Основные параметры')
        tabs.set_title(1, 'Отступы')
        
        # Кнопки действий
        buttons_box = HBox([
            self.process_button,
            self.save_button,
            self.save_params_button
        ])
        
        batch_box = VBox([self.batch_process_button])
        
        # Главный контейнер
        main_ui = VBox([
            image_selector,
            tabs,
            buttons_box,
            batch_box,
            self.output
        ])
        
        display(main_ui)

# Создаем и отображаем интерфейс
def create_normalizer_ui():
    normalizer = ImageNormalizer()
    normalizer.create_ui()
    return normalizer

# Вызываем функцию создания интерфейса
create_normalizer_ui()

## ОЧИСТКА БЭКА нейронкой

In [5]:
import os
import requests

# API ключ от remove.bg (бесплатная регистрация на remove.bg)
API_KEY = 'DLcQR2cxiDAes6J3WX8BZRd6'

# Пути к входной и выходной папкам
input_folder = 'final'
output_folder = 'final_transp'

# Создаем выходную папку, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Счетчики
total_processed = 0
total_errors = 0

# Перебираем все файлы во входной папке
for filename in os.listdir(input_folder):
    # Проверяем, что это изображение
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        # Полный путь к входному файлу
        input_path = os.path.join(input_folder, filename)
        
        # Полный путь к выходному файлу
        output_filename = os.path.splitext(filename)[0] + '.png'
        output_path = os.path.join(output_folder, output_filename)
        
        try:
            # Открываем файл для отправки
            with open(input_path, 'rb') as file:
                # Отправляем запрос на remove.bg
                response = requests.post(
                    'https://api.remove.bg/v1.0/removebg',
                    files={'image_file': file},
                    data={'size': 'auto'},
                    headers={'X-Api-Key': API_KEY}
                )
                
                # Проверяем успешность запроса
                if response.status_code == 200:
                    # Сохраняем результат
                    with open(output_path, 'wb') as out:
                        out.write(response.content)
                    print(f'Обработано изображение: {filename}')
                    total_processed += 1
                else:
                    print(f'Ошибка обработки {filename}: {response.text}')
                    total_errors += 1
        
        except Exception as e:
            print(f'Ошибка при обработке {filename}: {e}')
            total_errors += 1

print(f'\nОбработка завершена!')
print(f'Всего обработано изображений: {total_processed}')
print(f'Количество ошибок: {total_errors}')

Обработано изображение: assistant.png
Обработано изображение: boss.png
Обработано изображение: cowoker.png
Обработано изображение: daniel.png
Обработано изображение: director.png
Обработано изображение: dr_anderson.png
Обработано изображение: housekeeper.png
Обработано изображение: landon.png
Обработано изображение: linda.png
Обработано изображение: mark.png
Обработано изображение: mother.png
Обработано изображение: olga_makushenko.png
Обработано изображение: shadow.png
Обработано изображение: stranger.png
Обработано изображение: student1.png
Обработано изображение: student2.png
Обработано изображение: teen_daniel.png
Обработано изображение: tom.png
Обработано изображение: trainer.png
Обработано изображение: tv_host.png
Обработано изображение: voice.png
Обработано изображение: whiskers.png

Обработка завершена!
Всего обработано изображений: 22
Количество ошибок: 0


## Удаление бг-2

In [2]:
import os
from pathlib import Path
from rembg import remove
from PIL import Image
import concurrent.futures

def process_image(input_path, output_path):
    """
    Обрабатывает одно изображение: удаляет фон и сохраняет результат
    
    Параметры:
    input_path (str): Путь к исходному изображению
    output_path (str): Путь для сохранения результата
    """
    try:
        # Загружаем изображение
        input_img = Image.open(input_path)
        
        # Удаляем фон
        output_img = remove(input_img)
        
        # Создаем директорию для сохранения, если она не существует
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # Сохраняем результат
        output_img.save(output_path)
        
        print(f"Обработано: {input_path} -> {output_path}")
        return True
    except Exception as e:
        print(f"Ошибка при обработке {input_path}: {str(e)}")
        return False

def process_directory(input_dir, output_dir, max_workers=4):
    """
    Обрабатывает все изображения из входной директории
    
    Параметры:
    input_dir (str): Путь к директории с исходными изображениями
    output_dir (str): Путь к директории для сохранения результатов
    max_workers (int): Максимальное количество параллельных процессов
    """
    # Создаем выходную директорию, если она не существует
    os.makedirs(output_dir, exist_ok=True)
    
    # Получаем список всех файлов в директории
    input_files = []
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.webp']:
        input_files.extend(list(Path(input_dir).glob(ext)))
        input_files.extend(list(Path(input_dir).glob(ext.upper())))
    
    if not input_files:
        print(f"Изображения не найдены в директории {input_dir}")
        return
    
    print(f"Найдено {len(input_files)} изображений для обработки")
    
    # Подготавливаем пути для сохранения результатов
    tasks = []
    for input_path in input_files:
        # Сохраняем структуру директорий
        rel_path = input_path.relative_to(input_dir) if input_path.is_relative_to(input_dir) else Path(input_path.name)
        output_path = Path(output_dir) / rel_path
        
        # Если формат входного файла не PNG, меняем его на PNG для выходного файла
        if output_path.suffix.lower() != '.png':
            output_path = output_path.with_suffix('.png')
        
        tasks.append((str(input_path), str(output_path)))
    
    # Обрабатываем изображения в параллельных потоках
    successful = 0
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_file = {executor.submit(process_image, input_path, output_path): input_path 
                          for input_path, output_path in tasks}
        
        for future in concurrent.futures.as_completed(future_to_file):
            if future.result():
                successful += 1
    
    print(f"Обработка завершена. Успешно обработано {successful} из {len(tasks)} изображений.")
    print(f"Результаты сохранены в: {output_dir}")

if __name__ == "__main__":
    # Пути к директориям
    input_directory = "final"
    output_directory = "final_2"
    
    # Запускаем обработку
    process_directory(input_directory, output_directory)

Найдено 4 изображений для обработки
Обработано: final\13_sophisticated.png -> final_2\13_sophisticated.png
Обработано: final\13_sophisticated.png -> final_2\13_sophisticated.png
Обработано: final\curly.png -> final_2\curly.pngОбработано: final\curly.png -> final_2\curly.png

Обработка завершена. Успешно обработано 4 из 4 изображений.
Результаты сохранены в: final_2
